<a href="https://colab.research.google.com/github/nafiuopeyemi88/EDA_ViennaHospital/blob/main/extract_data_for_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap
!pip install geopandas
!pip install rasterstats
!pip install scipy
!pip install dask_geopandas

import ee
import geemap
import os
import math
import glob #reading and matching files
import numpy as np #array manipulation

import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd

import dask_geopandas





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.7 MB/s eta 0:00:00
  Attempting uninstall: dask
    Found existing installation: dask 2024.12.1
    Uninstalling dask-2024.12.1:
      Successfully uninstalled dask-2024.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2024.12.1 requires dask==2024.12.1, but you have dask 2025.2.0 which is incompatible.
dask-expr 1.1.21 requires dask==2024.12.1, but you have dask 2025.2.0 which is incompatible.
rapids-d

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-nafiuopeyemie')

## Read in Training data

In [ ]:
import geopandas as gpd

In [ ]:
tile_1 = gpd.read_file("farms.shp")
tile_1.shape

(8, 1)

In [ ]:
if tile_1.crs is None:
    tile_1.set_crs("epsg:4326", inplace=True)

In [ ]:
tile_1.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
# Read the shapefile
tile_1 = gpd.read_file("farms.shp")

# Set the current CRS if it's not defined
if tile_1.crs is None:
    # Replace 'epsg:YOUR_CURRENT_CRS' with the actual CRS of your shapefile
    tile_1.set_crs("epsg:YOUR_CURRENT_CRS", inplace=True)

# Now you can transform to the desired CRS
tile_1.to_crs(epsg=4326, inplace=True)

In [ ]:
tile_1.to_crs(epsg=4326, inplace=True)

In [ ]:
#add unique id to tile_1
tile_1['tile_id'] = range(1, len(tile_1) + 1)

In [ ]:
tile_1.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
tile_1.to_csv('tile_1.csv')

In [ ]:
def gdf_to_fc(gdf):
    features = []
    for _, row in gdf.iterrows():
        point = ee.Geometry.Point([row.geometry.x, row.geometry.y])
        feature = ee.Feature(point, row.drop('geometry').to_dict())
        features.append(feature)
    return ee.FeatureCollection(features)

fc = gdf_to_fc(tile_1)


In [ ]:
# Landsat Collections
L7_SR_T1 = ee.ImageCollection("LANDSAT/LE07/C02/T1")
L8_SR_T1 = ee.ImageCollection("LANDSAT/LC08/C02/T1")

# Cloud Masking Function
def cloudMask(image):
    qa = image.select('QA_PIXEL')
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)


# Rename Bands Function
def renameBands(image):
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'QA_PIXEL']
    new_bands = ['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa']
    return image.select(bands).rename(new_bands)


# l7coll =  ee.ImageCollection(L7_SR_T1)\
#                 .filterBounds(aoi)\
#                 .map(cloudMask)\
#                 .map(renameBandsL457)\
#                 .map(veg_indices_landsat)
# l8coll =  ee.ImageCollection(L8_SR_T1)\
#                 .filterBounds(aoi)\
#                 .map(cloudMask)\
#                 .map(renameBandsL8)\
#                 .map(veg_indices_landsat)

def veg_indices_landsat(image):

  SAVI = image.expression(
      '((NIR - RED) / (NIR + RED + 0.5)) * 1.5', {
          'NIR': image.select('NIR'),
          'RED': image.select('R')
      }).rename('SAVI')

  NDMI = image.expression(
      '(NIR - SWIR1) / (NIR + SWIR1)', {
          'NIR': image.select('NIR'),
          'SWIR1': image.select('SWIR1')
      }).rename('NDMI')

  NDWI = image.expression(
      '(GREEN - NIR) / (GREEN + NIR)', {
          'NIR': image.select('NIR'),
          'GREEN': image.select('G')
      }).rename('NDWI')

  NBR = image.expression(
      '(NIR - SWIR2) / (NIR + SWIR2)', {
          'NIR': image.select('NIR'),
          'SWIR2': image.select('SWIR2')
      }).rename('NBR')

  MSAVI = image.expression(
      '((2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED))) / 2)', {
          'NIR': image.select('NIR'),
          'RED': image.select('R')
      }).rename('MSAVI')
  image = image.addBands(ee.Image.cat([NDMI,NDWI, NBR, MSAVI, SAVI]))
  return image.select(['NDMI','NDWI', "NBR", "MSAVI", "SAVI"])



# Apply Preprocessing to Collections
l7coll = L7_SR_T1.map(cloudMask).map(renameBands).map(veg_indices_landsat)
l8coll = L8_SR_T1.map(cloudMask).map(renameBands).map(veg_indices_landsat)

L78 = l8coll.merge(l7coll)

# Temporal Range & Percentile
start_year = 2000
end_year = 2024
percentile = 25  # Change as needed

# Monthly Processing Function
def process_month(month):
    # Filter images for specific month and years
    monthly_images = L78.filter(ee.Filter.calendarRange(month, month, 'month')) \
                        .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
                        .filterBounds(fc)  # `fc` is your FeatureCollection (points)

    # Apply percentile reducer across all images
    monthly_percentile = monthly_images.reduce(ee.Reducer.percentile([percentile]))

    # Extract values for each point
    reduced_fc = monthly_percentile.reduceRegions(
        collection=fc,
        reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
        scale=30
    )

    # Add month info
    return reduced_fc.map(lambda feature: feature.set('Month', month))



monthly_results = ee.List.sequence(1, 12).map(process_month)
final_results = ee.FeatureCollection(monthly_results).flatten()

#


In [ ]:
#export it
task = ee.batch.Export.table.toDrive(
    collection= final_results,
    description='Landsat_indices_train_italy',
    folder='GEE_Exports',
    fileFormat='CSV'
)
task.start()


Climate dataset

In [ ]:


# Load the WorldClim dataset and select precipitation ('prec') band
world_clim = ee.ImageCollection("WORLDCLIM/V1/MONTHLY").select("prec")

# Define the FeatureCollection (replace with your own)


def process_month(month):
    """Processes a given month to compute mean precipitation across all years."""
    # Filter the image collection for the given month
    monthly_images = world_clim.filter(ee.Filter.eq('month', month))
    # Compute the mean precipitation across years
    monthly_mean = monthly_images.mean()

    # Ensure it has the band before using reduceRegions
    reduced_fc = monthly_mean.reduceRegions(
        collection=fc,
        reducer=ee.Reducer.mean(),
        scale=30  # Adjusted to match dataset resolution
    )

    # Add month information to each feature
    return reduced_fc.map(lambda feature: feature.set('Month', month))

# Process all months (1 to 12) and flatten results
monthly_results = ee.List.sequence(1, 12).map(process_month)
final_results = ee.FeatureCollection(monthly_results).flatten()





In [ ]:
#export it
task = ee.batch.Export.table.toDrive(
    collection= final_results,
    description='Monthly_precipitation_italy',
    folder='GEE_Exports',
    fileFormat='CSV'
)
task.start()

MERIT DEM STATIC DATA
hEIGHT ABOVE NEAREST NEIGHBOUR

In [ ]:
merit_dem = ee.Image("MERIT/Hydro/v1_0_1").select("hnd")
reduced_fc = merit_dem.reduceRegions(
        collection=fc,
        reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
        scale=30
    )

task = ee.batch.Export.table.toDrive(
    collection= reduced_fc,
    description='MERIT_height_above_nearest_neighbour_italy',
    folder='GEE_Exports',
    fileFormat='CSV'
)
task.start()

MERIT UPSTREAM AREA

In [ ]:
merit_dem = ee.Image("MERIT/Hydro/v1_0_1").select("upa")
reduced_fc = merit_dem.reduceRegions(
        collection=fc,
        reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
        scale=30
    )

task = ee.batch.Export.table.toDrive(
    collection= reduced_fc,
    description='MERIT_UPSTREAM_AREA_ITALY',
    folder='GEE_Exports',
    fileFormat='CSV'
)
task.start()

LANDSAT BAND INDICES

In [ ]:
# Landsat Collections
L7_SR_T1 = ee.ImageCollection("LANDSAT/LE07/C02/T1")
L8_SR_T1 = ee.ImageCollection("LANDSAT/LC08/C02/T1")

# Cloud Masking Function
def cloudMask(image):
    qa = image.select('QA_PIXEL')
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

def veg_indices_landsat(image):
  NDVI = image.expression(
        '((NIR - RED) / (NIR + RED))', {
            'NIR': image.select('NIR'),  # Select the near-infrared (NIR) band (B8)
            'RED': image.select('R')   # Select the red band (B4)
        }).rename("NDVI")  # Rename the resulting band as NDVI
  # Compute the EVI using an expression
  EVI = image.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {

          'NIR': image.select('NIR').divide(10000),  # Select and normalize the near-infrared (NIR) band
          'RED': image.select('R').divide(10000),  # Select and normalize the red band
          'BLUE': image.select('B').divide(10000)  # Select and normalize the blue band
      }).rename("EVI")  # Rename the resulting band as EVI

  SAVI = image.expression(
      '((NIR - RED) / (NIR + RED + 0.5)) * 1.5', {
          'NIR': image.select('NIR'),
          'RED': image.select('R')
      }).rename('SAVI')

  BSI = image.expression(
      '((Swir + Red) - (Nir + Blue)) / ((Swir + Red) + (Nir + Blue))', {
          'Swir': image.select('SWIR1'),
          'Nir': image.select('NIR'),
          'Red': image.select('R'),
          'Blue': image.select('B')
      }
  )
  image = image.addBands(ee.Image.cat([NDVI,EVI, BSI]))
  return image#.select(['NDVI','EVI'])


# Rename Bands Function
def renameBands(image):
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'QA_PIXEL']
    new_bands = ['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa']
    return image.select(bands).rename(new_bands)


# l7coll =  ee.ImageCollection(L7_SR_T1)\
#                 .filterBounds(aoi)\
#                 .map(cloudMask)\
#                 .map(renameBandsL457)\
#                 .map(veg_indices_landsat)
# l8coll =  ee.ImageCollection(L8_SR_T1)\
#                 .filterBounds(aoi)\
#                 .map(cloudMask)\
#                 .map(renameBandsL8)\
#                 .map(veg_indices_landsat)



# Apply Preprocessing to Collections
l7coll = L7_SR_T1.map(cloudMask).map(renameBands).map(veg_indices_landsat)
l8coll = L8_SR_T1.map(cloudMask).map(renameBands).map(veg_indices_landsat)

L78 = l8coll.merge(l7coll)

# Temporal Range & Percentile
start_year = 2000
end_year = 2024
percentile = 25  # Change as needed

# Monthly Processing Function
def process_month(month):
    # Filter images for specific month and years
    monthly_images = L78.filter(ee.Filter.calendarRange(month, month, 'month')) \
                        .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
                        .filterBounds(fc)  # `fc` is your FeatureCollection (points)

    # Apply percentile reducer across all images
    monthly_percentile = monthly_images.reduce(ee.Reducer.percentile([percentile]))

    # Extract values for each point
    reduced_fc = monthly_percentile.reduceRegions(
        collection=fc,
        reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
        scale=30
    )

    # Add month info
    return reduced_fc.map(lambda feature: feature.set('Month', month))



monthly_results = ee.List.sequence(1, 12).map(process_month)
final_results = ee.FeatureCollection(monthly_results).flatten()

#

In [ ]:
task = ee.batch.Export.table.toDrive(
    collection=final_results,
    description='landsat_Export_bands_indices_italy',
    folder='GEE_Exports',
    fileFormat='CSV'
)
task.start()

MODIS EVI

In [ ]:



# Area of Interest (AOI)
aoi = ee.Geometry.Point(25.8544, -18.08874)

# MODIS Collections
modis_evi = ee.ImageCollection('MODIS/061/MOD13Q1').select('EVI')
modis_lst = ee.ImageCollection("MODIS/061/MOD11A1").select('LST_Day_1km', 'QC_Night')

# Resample Function
def resample_to_30m(image):
    return image.resample('bicubic').reproject(crs='EPSG:4326', scale=30)



# Years of interest
start_year = 2000
end_year = 2024
percentile = [25]

def process_month(month):
    # Filter images for specific month and years
    monthly_images = modis_evi.filter(ee.Filter.calendarRange(month, month, 'month')) \
                        .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
                        .filterBounds(fc)

    #resampled_image = monthly_images.map(resample_to_30m)
    monthly_percentile = monthly_images.reduce(ee.Reducer.percentile(percentile))
    resampled_image = resample_to_30m(monthly_percentile)
    # Extract values for each point
    reduced_fc = resampled_image.reduceRegions(
        collection=fc,
        reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
        scale=30
    )
    return reduced_fc.map(lambda feature: feature.set('Month', month))




monthly_results = ee.List.sequence(1, 12).map(process_month)
final_results = ee.FeatureCollection(monthly_results).flatten()


In [ ]:
task = ee.batch.Export.table.toDrive(
    collection=final_results,
    description='modis_evi_italy',
    folder='GEE_Exports',
    fileFormat='CSV'
)
task.start()

Modis Daytime Lst

In [ ]:
modis_lst = ee.ImageCollection("MODIS/061/MOD11A1").select('LST_Day_1km')

# Resample Function
def resample_to_30m(image):
    return image.resample('bicubic').reproject(crs='EPSG:4326', scale=30)



# Years of interest
start_year = 2000
end_year = 2024
percentile = [5, 50, 95]

def process_month(month):
    # Filter images for specific month and years
    monthly_images = modis_lst.filter(ee.Filter.calendarRange(month, month, 'month')) \
                        .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
                        .filterBounds(fc)

    #resampled_image = monthly_images.map(resample_to_30m)
    monthly_percentile = monthly_images.reduce(ee.Reducer.percentile(percentile))
    resampled_image = resample_to_30m(monthly_percentile)
    # Extract values for each point
    reduced_fc = resampled_image.reduceRegions(
        collection=fc,
        reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
        scale=30
    )
    return reduced_fc.map(lambda feature: feature.set('Month', month))




monthly_results = ee.List.sequence(1, 12).map(process_month)
final_results = ee.FeatureCollection(monthly_results).flatten()


In [ ]:
task = ee.batch.Export.table.toDrive(
    collection=final_results,
    description='modis_lst_daytime_italy',
    folder='GEE_Exports',
    fileFormat='CSV'
)
task.start()

In [ ]:
modis_lst = ee.ImageCollection("MODIS/061/MOD11A1").select('LST_Night_1km')

# Resample Function
def resample_to_30m(image):
    return image.resample('bicubic').reproject(crs='EPSG:4326', scale=30)



# Years of interest
start_year = 2000
end_year = 2024
percentile = [5, 50, 95]

def process_month(month):
    # Filter images for specific month and years
    monthly_images = modis_lst.filter(ee.Filter.calendarRange(month, month, 'month')) \
                        .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
                        .filterBounds(fc)

    #resampled_image = monthly_images.map(resample_to_30m)
    monthly_percentile = monthly_images.reduce(ee.Reducer.percentile(percentile))
    resampled_image = resample_to_30m(monthly_percentile)
    # Extract values for each point
    reduced_fc = resampled_image.reduceRegions(
        collection=fc,
        reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
        scale=30
    )
    return reduced_fc.map(lambda feature: feature.set('Month', month))




monthly_results = ee.List.sequence(1, 12).map(process_month)
final_results = ee.FeatureCollection(monthly_results).flatten()


In [ ]:
task = ee.batch.Export.table.toDrive(
    collection=final_results,
    description='modis_lst_nightime_italy',
    folder='GEE_Exports',
    fileFormat='CSV'
)
task.start()

Soil Properties dataset

In [ ]:
#'OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02'

soil_dataset = ee.Image('OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02').select('b0', 'b10', 'b30', 'b60')

def resample_to_30m(image):
    return image.resample('bicubic').reproject(crs='EPSG:4326', scale=30)

resampled_image = resample_to_30m(soil_dataset)

reduced_fc = resampled_image.reduceRegions(
        collection=fc,
        reducer=ee.Reducer.mean(),
        scale=30
    )


task = ee.batch.Export.table.toDrive(
    collection=reduced_fc,
    description='soil_organic_carbon_250m_ITALY',
    folder='GEE_Exports',
    fileFormat='CSV'
)
task.start()

SRTM

In [ ]:
#'OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02'

elevation_dataset = ee.Image("USGS/SRTMGL1_003").select('elevation')

reduced_fc = elevation_dataset.reduceRegions(
        collection=fc,
        reducer=ee.Reducer.mean(),
        scale=30
    )


task = ee.batch.Export.table.toDrive(
    collection=reduced_fc,
    description='elevation_ITALY',
    folder='GEE_Exports',
    fileFormat='CSV'
)
task.start()

Surface Reflectance

In [ ]:
# Landsat Collections
L7_SR_T1 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
L8_SR_T1 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

# Cloud Masking Function
def cloudMask(image):
    qa = image.select('QA_PIXEL')
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  return image.addBands(optical_bands, None, True)

def veg_indices_landsat(image):
  NDVI = image.expression(
        '((NIR - RED) / (NIR + RED))', {
            'NIR': image.select('NIR'),  # Select the near-infrared (NIR) band (B8)
            'RED': image.select('R')   # Select the red band (B4)
        }).rename("NDVI")  # Rename the resulting band as NDVI
  # Compute the EVI using an expression
  EVI = image.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {

          'NIR': image.select('NIR'),
          'RED': image.select('R'),
          'BLUE': image.select('B')
      }).rename("EVI")  # Rename the resulting band as EVI

  SAVI = image.expression(
      '((NIR - RED) / (NIR + RED + 0.5)) * 1.5', {
          'NIR': image.select('NIR'),
          'RED': image.select('R')
      }).rename('SAVI')

  BSI = image.expression(
      '((Swir + Red) - (Nir + Blue)) / ((Swir + Red) + (Nir + Blue))', {
          'Swir': image.select('SWIR1'),
          'Nir': image.select('NIR'),
          'Red': image.select('R'),
          'Blue': image.select('B')
      }
  )
  SAVI = image.expression(
      '((NIR - RED) / (NIR + RED + 0.5)) * 1.5', {
          'NIR': image.select('NIR'),
          'RED': image.select('R')
      }).rename('SAVI')

  NDMI = image.expression(
      '(NIR - SWIR1) / (NIR + SWIR1)', {
          'NIR': image.select('NIR'),
          'SWIR1': image.select('SWIR1')
      }).rename('NDMI')

  NDWI = image.expression(
      '(GREEN - NIR) / (GREEN + NIR)', {
          'NIR': image.select('NIR'),
          'GREEN': image.select('G')
      }).rename('NDWI')

  NBR = image.expression(
      '(NIR - SWIR2) / (NIR + SWIR2)', {
          'NIR': image.select('NIR'),
          'SWIR2': image.select('SWIR2')
      }).rename('NBR')

  MSAVI = image.expression(
      '((2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED))) / 2)', {
          'NIR': image.select('NIR'),
          'RED': image.select('R')
      }).rename('MSAVI')

  image = image.addBands(ee.Image.cat([NDVI, SAVI, NDMI, NDWI, NBR, MSAVI]))
  return image#.select(['NDVI','SAVI', 'NDMI', 'NDWI', 'NBR', 'MSAVI'])


# Rename Bands Function
def renameBands(image):
    bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']
    new_bands = ['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa']
    return image.select(bands).rename(new_bands)



# l7coll =  ee.ImageCollection(L7_SR_T1)\
#                 .filterBounds(aoi)\
#                 .map(cloudMask)\
#                 .map(renameBandsL457)\
#                 .map(veg_indices_landsat)
# l8coll =  ee.ImageCollection(L8_SR_T1)\
#                 .filterBounds(aoi)\
#                 .map(cloudMask)\
#                 .map(renameBandsL8)\
#                 .map(veg_indices_landsat)



# Apply Preprocessing to Collections
#comment out vegetation indices for band by band processing
l7coll = L7_SR_T1.map(cloudMask).map(apply_scale_factors).map(renameBands)#.map(veg_indices_landsat)
l8coll = L8_SR_T1.map(cloudMask).map(apply_scale_factors).map(renameBands)#.map(veg_indices_landsat)

L78 = l8coll.merge(l7coll)

# Temporal Range & Percentile
start_year = 2000
end_year = 2024
percentile = 25  # Change as needed

# Define the intervals in day-of-year (DOY) format
march_interval = ee.Filter.dayOfYear(80, 96)   # March 21 - April 5 - this is .6
june_interval = ee.Filter.dayOfYear(176, 192)  # this is .12
september_interval = ee.Filter.dayOfYear(256, 272)  # September 13 - .17
november_interval = ee.Filter.dayOfYear(336, 352) #.22
#the code will run for each interval seperately


# Temporal Range & Percentile
start_year = 2000
end_year = 2024
percentile = 25  # Change as needed

# Filter images for the given year range
L78_filtered = L78.filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
                  .filter(ee.Filter.Or(november_interval)) \
                  .filterBounds(fc)  # `fc` is your FeatureCollection

# Apply percentile reducer
percentile_image = L78_filtered.reduce(ee.Reducer.percentile([percentile]))

# Reduce over FeatureCollection
reduced_fc = percentile_image.reduceRegions(
    collection=fc,
    reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
    scale=30
)

# Return feature collection with an identifier
reduced_fc = reduced_fc.map(lambda feature: feature.set('Interval', 'nov_dec'))



In [ ]:
#export it
task = ee.batch.Export.table.toDrive(
    collection= reduced_fc,
    description='Landsat_bands_nov_dec_italy',
    folder='GEE_Exports',
    fileFormat='CSV'
)
task.start()


Yearly Landsat

In [ ]:
L8_SR_T1 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

L7_SR_T1 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")

def renameBands(image):
    bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']
    new_bands = ['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa']
    return image.select(bands).rename(new_bands)


def cloudMask(image):
    qa = image.select('QA_PIXEL')
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  return image.addBands(optical_bands, None, True)

start_date = '2024-01-01'
end_date = '2024-12-31'

L8_filtered = L8_SR_T1.filter(ee.Filter.date(start_date, end_date)) \
                  .filterBounds(fc)  # `fc` is your FeatureCollection

l8coll = L8_filtered.map(cloudMask).map(apply_scale_factors).map(renameBands)

reduced_image = l8coll.mean()


reduced_fc = reduced_image.reduceRegions(
    collection=fc,
    reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
    scale=30
)

In [ ]:
#export it
task = ee.batch.Export.table.toDrive(
    collection= reduced_fc,
    description='Landsat_bands_2024_italy',
    folder='GEE_Exports',
    fileFormat='CSV'
)
task.start()


## Processing

In [ ]:
#yearly
import pandas as pd
df_2000 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/Landsat_bands_2014_italy.csv')
df_2014 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/Landsat_bands_2014_italy.csv')
df_2024 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/Landsat_bands_2024_italy.csv')


In [ ]:
df_2000.columns

Index(['system:index', 'B', 'G', 'Id', 'NIR', 'R', 'SWIR1', 'SWIR2',
       'pixel_qa', 'tile_id', '.geo'],
      dtype='object')

In [ ]:
def process_df(df, year):
  df = df.drop(['.geo', 'system:index'], axis = 1)
  df.columns = [col + f'_landsat{year}' if col != 'tile_id' else col for col in df.columns]
  return df

In [ ]:
#use map to apply the process_df function to the dataframes
df_2000 = process_df(df_2000, 2000)

In [ ]:
df_2014 = process_df(df_2014, 2014)
df_2024 = process_df(df_2024, 2024)

In [ ]:
#merge df_2000 on tile_id
df_merged = pd.merge(df_2000, df_2014, on='tile_id', how='outer')

In [ ]:
df_merged_yearly = pd.merge(df_merged, df_2024, on='tile_id', how='outer')

In [ ]:
df_merged_yearly.to_csv('/content/drive/MyDrive/GEE_Exports/train_data/Landsat_bands_yearly_italy.csv')

## toa cleaning

In [ ]:
landsat_bd_indices = pd.read_csv('/content/drive/MyDrive/GEE_Exports/landsat_Export_bands_indices_italy.csv')

In [ ]:
landsat_bd_indices.columns

Index(['system:index', 'B_p25', 'EVI_p25', 'G_p25', 'Id', 'Month', 'NDVI_p25',
       'NIR_p25', 'R_p25', 'SWIR1_1_p25', 'SWIR1_p25', 'SWIR2_p25',
       'pixel_qa_p25', 'tile_id', '.geo'],
      dtype='object')

In [ ]:
landsat_bd_indices = landsat_bd_indices.drop(['.geo', 'system:index', 'pixel_qa_p25'], axis = 1)

In [ ]:
month_map = {
    1.0: "Jan", 2.0: "Feb", 3.0: "Mar", 4.0: "Apr", 5.0: "May", 6.0: "Jun",
    7.0: "Jul", 8.0: "Aug", 9.0: "Sep", 10.0: "Oct", 11.0: "Nov", 12.0: "Dec"
}

landsat_bd_indices["Month"] = landsat_bd_indices["Month"].map(month_map)

In [ ]:
pivoted_df = landsat_bd_indices.pivot(index='tile_id', columns='Month')
pivoted_df

B_p25                                                            ...  \
Month     Apr    Aug   Dec   Feb   Jan    Jul    Jun   Mar    May   Nov  ...   
tile_id                                                                  ...   
1        82.0   88.0  48.0  51.0  53.0   94.0   95.0  69.5   85.0  56.0  ...   
2        83.0   90.0  49.0  54.0  52.0   94.0   97.0  68.0   85.0  55.0  ...   
3        80.0   91.5  50.0  52.0  54.0   97.0   99.0  67.5   84.5  59.0  ...   
4        81.0   91.5  49.0  52.0  53.0   96.0   99.0  66.5   84.0  57.0  ...   
5        81.0   92.0  51.0  52.0  53.0   97.5  100.0  67.5   83.0  59.0  ...   
...       ...    ...   ...   ...   ...    ...    ...   ...    ...   ...  ...   
27885    84.0   98.0  52.0  55.0  57.0  108.0  101.0  68.0   89.0  60.0  ...   
27886    84.0   98.0  52.0  55.0  57.0  108.0  101.0  68.0   89.0  60.0  ...   
27887    84.0   99.0  54.0  54.5  58.0  107.0  102.0  68.0   86.0  60.0  ...   
27888    83.0  100.5  52.0  54.5  57.0  108.0  100.0  68.5   88.0  57.5  ...   
27889    91.0  105.0  56.0  63.0  60.0  112.0  109.0  77.0  100.0  62.0  ...   

        SWIR2_p25                                                          
Month         Dec   Feb   Jan    Jul    Jun   Mar   May   Nov   Oct   Sep  
tile_id                                                                    
1            21.0  19.0  24.5   96.0   95.0  42.5  74.0  28.0  37.0  55.0  
2            25.0  20.0  26.0  100.0   98.0  41.5  71.0  28.0  43.0  63.0  
3            28.0  21.0  30.0  106.0   99.0  32.5  58.5  34.5  45.0  73.0  
4            27.0  21.0  26.5  106.0  108.0  31.5  54.5  38.0  47.0  80.0  
5            26.0  22.0  25.0  107.5   98.0  32.5  51.0  37.5  42.0  82.0  
...           ...   ...   ...    ...    ...   ...   ...   ...   ...   ...  
27885        25.0  29.0  29.0  106.0   97.0  34.0  65.5  33.0  27.0  83.5  
27886        25.0  29.0  29.0  106.0   97.0  34.0  65.5  33.0  27.0  83.5  
27887        24.0  27.5  29.0  109.0  102.0  34.0  72.0  33.0  28.0  82.5  
27888        25.0  27.0  27.0  108.0  102.0  33.5  66.5  30.5  28.0  83.0  
27889        34.0  42.0  36.0  118.0  115.0  61.0  89.0  38.5  52.0  80.0  

[27889 rows x 120 columns]

In [ ]:
pivoted_df.columns = [f"{col[0].lower()}_{col[1].lower()}" for col in pivoted_df.columns]
pivoted_df.reset_index(inplace=True)
pivoted_df

,tile_id,b_p25_apr,b_p25_aug,b_p25_dec,b_p25_feb,b_p25_jan,b_p25_jul,b_p25_jun,b_p25_mar,b_p25_may,...,swir2_p25_dec,swir2_p25_feb,swir2_p25_jan,swir2_p25_jul,swir2_p25_jun,swir2_p25_mar,swir2_p25_may,swir2_p25_nov,swir2_p25_oct,swir2_p25_sep
0,1,82.0,88.0,48.0,51.0,53.0,94.0,95.0,69.5,85.0,...,21.0,19.0,24.5,96.0,95.0,42.5,74.0,28.0,37.0,55.0
1,2,83.0,90.0,49.0,54.0,52.0,94.0,97.0,68.0,85.0,...,25.0,20.0,26.0,100.0,98.0,41.5,71.0,28.0,43.0,63.0
2,3,80.0,91.5,50.0,52.0,54.0,97.0,99.0,67.5,84.5,...,28.0,21.0,30.0,106.0,99.0,32.5,58.5,34.5,45.0,73.0
3,4,81.0,91.5,49.0,52.0,53.0,96.0,99.0,66.5,84.0,...,27.0,21.0,26.5,106.0,108.0,31.5,54.5,38.0,47.0,80.0
4,5,81.0,92.0,51.0,52.0,53.0,97.5,100.0,67.5,83.0,...,26.0,22.0,25.0,107.5,98.0,32.5,51.0,37.5,42.0,82.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27884,27885,84.0,98.0,52.0,55.0,57.0,108.0,101.0,68.0,89.0,...,25.0,29.0,29.0,106.0,97.0,34.0,65.5,33.0,27.0,83.5
27885,27886,84.0,98.0,52.0,55.0,57.0,108.0,101.0,68.0,89.0,...,25.0,29.0,29.0,106.0,97.0,34.0,65.5,33.0,27.0,83.5
27886,27887,84.0,99.0,54.0,54.5,58.0,107.0,102.0,68.0,86.0,...,24.0,27.5,29.0,109.0,102.0,34.0,72.0,33.0,28.0,82.5
27887,27888,83.0,100.5,52.0,54.5,57.0,108.0,100.0,68.5,88.0,...,25.0,27.0,27.0,108.0,102.0,33.5,66.5,30.5,28.0,83.0


In [ ]:
pivoted_df.columns.values

array(['tile_id', 'b_p25_apr', 'b_p25_aug', 'b_p25_dec', 'b_p25_feb',
       'b_p25_jan', 'b_p25_jul', 'b_p25_jun', 'b_p25_mar', 'b_p25_may',
       'b_p25_nov', 'b_p25_oct', 'b_p25_sep', 'evi_p25_apr',
       'evi_p25_aug', 'evi_p25_dec', 'evi_p25_feb', 'evi_p25_jan',
       'evi_p25_jul', 'evi_p25_jun', 'evi_p25_mar', 'evi_p25_may',
       'evi_p25_nov', 'evi_p25_oct', 'evi_p25_sep', 'g_p25_apr',
       'g_p25_aug', 'g_p25_dec', 'g_p25_feb', 'g_p25_jan', 'g_p25_jul',
       'g_p25_jun', 'g_p25_mar', 'g_p25_may', 'g_p25_nov', 'g_p25_oct',
       'g_p25_sep', 'ndvi_p25_apr', 'ndvi_p25_aug', 'ndvi_p25_dec',
       'ndvi_p25_feb', 'ndvi_p25_jan', 'ndvi_p25_jul', 'ndvi_p25_jun',
       'ndvi_p25_mar', 'ndvi_p25_may', 'ndvi_p25_nov', 'ndvi_p25_oct',
       'ndvi_p25_sep', 'nir_p25_apr', 'nir_p25_aug', 'nir_p25_dec',
       'nir_p25_feb', 'nir_p25_jan', 'nir_p25_jul', 'nir_p25_jun',
       'nir_p25_mar', 'nir_p25_may', 'nir_p25_nov', 'nir_p25_oct',
       'nir_p25_sep', 'r_p25_apr', 'r_p25_

In [ ]:
pivoted_df.drop(['id_apr', 'id_aug', 'id_dec', 'id_feb', 'id_jan',
       'id_jul', 'id_jun', 'id_mar', 'id_may', 'id_nov', 'id_oct',
       'id_sep',], axis = 1, inplace = True)

In [ ]:
pivoted_df.to_csv('/content/drive/MyDrive/GEE_Exports/train_data/landsat_Export_toa_bands_italy.csv')

## TOA INDICES

In [ ]:
landsat_indices_toa = pd.read_csv('/content/drive/MyDrive/GEE_Exports/Landsat_indices_train_italy.csv')
landsat_indices_toa.columns

Index(['system:index', 'Id', 'MSAVI_p25', 'Month', 'NBR_p25', 'NDMI_p25',
       'NDWI_p25', 'SAVI_p25', 'tile_id', '.geo'],
      dtype='object')

In [ ]:
landsat_indices_toa = landsat_indices_toa.drop(['.geo', 'system:index', 'Id'], axis = 1)


In [ ]:
landsat_indices_toa["Month"] = landsat_indices_toa["Month"].map(month_map)

In [ ]:
pivoted_df = landsat_indices_toa.pivot(index='tile_id', columns='Month')
pivoted_df

MSAVI_p25                                                              \
Month         Apr       Aug       Dec       Feb       Jan       Jul       Jun   
tile_id                                                                         
1       -0.087616 -0.367959 -0.059276 -0.087185 -0.070645 -0.315829 -0.226586   
2       -0.067565 -0.371026 -0.055409 -0.092052 -0.074994 -0.378516 -0.288536   
3       -0.115100 -0.544449 -0.016804 -0.092438 -0.100931 -0.475958 -0.426833   
4       -0.143638 -0.575870 -0.055590 -0.071304 -0.106483 -0.540054 -0.501488   
5       -0.145001 -0.638391 -0.039329 -0.083339 -0.101983 -0.578671 -0.525294   
...           ...       ...       ...       ...       ...       ...       ...   
27885   -0.131010 -0.713596 -0.043935 -0.058092 -0.063919 -0.626463 -0.578049   
27886   -0.131010 -0.713596 -0.043935 -0.058092 -0.063919 -0.626463 -0.578049   
27887   -0.137665 -0.695914 -0.056045 -0.046507 -0.063113 -0.663895 -0.594081   
27888   -0.127710 -0.698829 -0.048699 -0.040894 -0.064471 -0.655715 -0.594081   
27889   -0.084805 -0.551022 -0.020199  0.000124 -0.018118 -0.516751 -0.557079   

                                       ...  SAVI_p25                      \
Month         Mar       May       Nov  ...       Dec       Feb       Jan   
tile_id                                ...                                 
1        0.000532 -0.124207 -0.049028  ... -0.043179 -0.062660 -0.051177   
2       -0.025529 -0.105198 -0.049340  ... -0.040438 -0.066004 -0.054215   
3       -0.035128 -0.150918 -0.027019  ... -0.012552 -0.066268 -0.072065   
4       -0.066835 -0.129430 -0.050942  ... -0.040566 -0.051639 -0.075828   
5       -0.048898 -0.127869 -0.065365  ... -0.028929 -0.060006 -0.072779   
...           ...       ...       ...  ...       ...       ...       ...   
27885   -0.060183 -0.366698 -0.021619  ... -0.032432 -0.042340 -0.046457   
27886   -0.060183 -0.366698 -0.021619  ... -0.032432 -0.042340 -0.046457   
27887   -0.058362 -0.454986 -0.032729  ... -0.041096 -0.034071 -0.045889   
27888   -0.060867 -0.472567 -0.034070  ... -0.035658 -0.030087 -0.047120   
27889   -0.038572 -0.418115  0.015893  ... -0.014999  0.000093 -0.013467   

                                                                               
Month         Jul       Jun       Mar       May       Nov       Oct       Sep  
tile_id                                                                        
1       -0.205607 -0.153355  0.000465 -0.088000 -0.035892  0.000000 -0.246057  
2       -0.240000 -0.189873 -0.018894 -0.075209 -0.036115  0.010309 -0.257880  
3       -0.289817 -0.264901 -0.025891 -0.105522 -0.020035 -0.027586 -0.326848  
4       -0.320611 -0.302158 -0.048410 -0.091380 -0.037326 -0.072165 -0.343278  
5       -0.338102 -0.313346 -0.035805 -0.090566 -0.047548 -0.074205 -0.354037  
...           ...       ...       ...       ...       ...       ...       ...  
27885   -0.359712 -0.339901 -0.043820 -0.233304 -0.016041 -0.085020 -0.396743  
27886   -0.359712 -0.339901 -0.043820 -0.233304 -0.016041 -0.085020 -0.396743  
27887   -0.375824 -0.345324 -0.042532 -0.279452 -0.024153 -0.090566 -0.392793  
27888   -0.372365 -0.345324 -0.044299 -0.288136 -0.025182 -0.094862 -0.393586  
27889   -0.309278 -0.328467 -0.028382 -0.260546  0.012020  0.000000 -0.324952  

[27889 rows x 60 columns]

In [ ]:
pivoted_df.columns = [f"{col[0].lower()}_{col[1].lower()}" for col in pivoted_df.columns]
pivoted_df.reset_index(inplace=True)
pivoted_df

,tile_id,msavi_p25_apr,msavi_p25_aug,msavi_p25_dec,msavi_p25_feb,msavi_p25_jan,msavi_p25_jul,msavi_p25_jun,msavi_p25_mar,msavi_p25_may,...,savi_p25_dec,savi_p25_feb,savi_p25_jan,savi_p25_jul,savi_p25_jun,savi_p25_mar,savi_p25_may,savi_p25_nov,savi_p25_oct,savi_p25_sep
0,1,-0.087616,-0.367959,-0.059276,-0.087185,-0.070645,-0.315829,-0.226586,0.000532,-0.124207,...,-0.043179,-0.062660,-0.051177,-0.205607,-0.153355,0.000465,-0.088000,-0.035892,0.000000,-0.246057
1,2,-0.067565,-0.371026,-0.055409,-0.092052,-0.074994,-0.378516,-0.288536,-0.025529,-0.105198,...,-0.040438,-0.066004,-0.054215,-0.240000,-0.189873,-0.018894,-0.075209,-0.036115,0.010309,-0.257880
2,3,-0.115100,-0.544449,-0.016804,-0.092438,-0.100931,-0.475958,-0.426833,-0.035128,-0.150918,...,-0.012552,-0.066268,-0.072065,-0.289817,-0.264901,-0.025891,-0.105522,-0.020035,-0.027586,-0.326848
3,4,-0.143638,-0.575870,-0.055590,-0.071304,-0.106483,-0.540054,-0.501488,-0.066835,-0.129430,...,-0.040566,-0.051639,-0.075828,-0.320611,-0.302158,-0.048410,-0.091380,-0.037326,-0.072165,-0.343278
4,5,-0.145001,-0.638391,-0.039329,-0.083339,-0.101983,-0.578671,-0.525294,-0.048898,-0.127869,...,-0.028929,-0.060006,-0.072779,-0.338102,-0.313346,-0.035805,-0.090566,-0.047548,-0.074205,-0.354037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27884,27885,-0.131010,-0.713596,-0.043935,-0.058092,-0.063919,-0.626463,-0.578049,-0.060183,-0.366698,...,-0.032432,-0.042340,-0.046457,-0.359712,-0.339901,-0.043820,-0.233304,-0.016041,-0.085020,-0.396743
27885,27886,-0.131010,-0.713596,-0.043935,-0.058092,-0.063919,-0.626463,-0.578049,-0.060183,-0.366698,...,-0.032432,-0.042340,-0.046457,-0.359712,-0.339901,-0.043820,-0.233304,-0.016041,-0.085020,-0.396743
27886,27887,-0.137665,-0.695914,-0.056045,-0.046507,-0.063113,-0.663895,-0.594081,-0.058362,-0.454986,...,-0.041096,-0.034071,-0.045889,-0.375824,-0.345324,-0.042532,-0.279452,-0.024153,-0.090566,-0.392793
27887,27888,-0.127710,-0.698829,-0.048699,-0.040894,-0.064471,-0.655715,-0.594081,-0.060867,-0.472567,...,-0.035658,-0.030087,-0.047120,-0.372365,-0.345324,-0.044299,-0.288136,-0.025182,-0.094862,-0.393586


In [ ]:
pivoted_df.columns.values

array(['tile_id', 'msavi_p25_apr', 'msavi_p25_aug', 'msavi_p25_dec',
       'msavi_p25_feb', 'msavi_p25_jan', 'msavi_p25_jul', 'msavi_p25_jun',
       'msavi_p25_mar', 'msavi_p25_may', 'msavi_p25_nov', 'msavi_p25_oct',
       'msavi_p25_sep', 'nbr_p25_apr', 'nbr_p25_aug', 'nbr_p25_dec',
       'nbr_p25_feb', 'nbr_p25_jan', 'nbr_p25_jul', 'nbr_p25_jun',
       'nbr_p25_mar', 'nbr_p25_may', 'nbr_p25_nov', 'nbr_p25_oct',
       'nbr_p25_sep', 'ndmi_p25_apr', 'ndmi_p25_aug', 'ndmi_p25_dec',
       'ndmi_p25_feb', 'ndmi_p25_jan', 'ndmi_p25_jul', 'ndmi_p25_jun',
       'ndmi_p25_mar', 'ndmi_p25_may', 'ndmi_p25_nov', 'ndmi_p25_oct',
       'ndmi_p25_sep', 'ndwi_p25_apr', 'ndwi_p25_aug', 'ndwi_p25_dec',
       'ndwi_p25_feb', 'ndwi_p25_jan', 'ndwi_p25_jul', 'ndwi_p25_jun',
       'ndwi_p25_mar', 'ndwi_p25_may', 'ndwi_p25_nov', 'ndwi_p25_oct',
       'ndwi_p25_sep', 'savi_p25_apr', 'savi_p25_aug', 'savi_p25_dec',
       'savi_p25_feb', 'savi_p25_jan', 'savi_p25_jul', 'savi_p25_jun',
       'sav

In [ ]:
pivoted_df.to_csv('/content/drive/MyDrive/GEE_Exports/train_data/landsat_Export_indices_italy.csv')

## Surface Reflectance

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/Landsat_bands_aug_sep_italy.csv')
df2 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/Landsat_bands_nov_dec_italy.csv')
df3 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/Landsat_bands_jun_july_italy.csv')
df4 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/Landsat_bands_mar_apr_italy.csv')


In [ ]:
df1.columns

Index(['system:index', 'B_p25', 'G_p25', 'Id', 'Interval', 'NIR_p25', 'R_p25',
       'SWIR1_p25', 'SWIR2_p25', 'pixel_qa_p25', 'tile_id', '.geo'],
      dtype='object')

In [ ]:
#aug, un, mar, nov

In [ ]:
def process_df(df, month):
  df = df.drop(['.geo', 'system:index', 'Interval', 'Id', 'pixel_qa_p25'], axis = 1)
  df.columns = [col + f'_{month}' if col != 'tile_id' else col for col in df.columns]
  return df

In [ ]:
df1 = process_df(df1, 'aug')
df2 = process_df(df2, 'nov')
df3 = process_df(df3, 'jun')
df4 = process_df(df4, 'mar')

In [ ]:
df1.columns

Index(['B_p25_aug', 'G_p25_aug', 'NIR_p25_aug', 'R_p25_aug', 'SWIR1_p25_aug',
       'SWIR2_p25_aug', 'tile_id'],
      dtype='object')

In [ ]:

dfs = [df1, df2, df3, df4]
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = merged_df.merge(df, on='tile_id', how='outer')

In [ ]:
merged_df.to_csv('/content/drive/MyDrive/GEE_Exports/train_data/Landsat_bands_surface_total_italy.csv')

## Indices Surface Reflectance

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/Landsat_indices_jun_july_italy.csv')
df2 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/Landsat_indices_mar_apr_italy.csv')
df3 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/Landsat_indices_aug_sep_italy.csv')
df4 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/Landsat_indices_oct_nov_italy.csv')



In [ ]:
def process_df(df, month):
  df = df.drop(['.geo', 'system:index', 'Interval', 'Id'], axis = 1)
  df.columns = [col + f'_{month}' if col != 'tile_id' else col for col in df.columns]
  return df

In [ ]:
df3 = process_df(df3, 'aug')
df2 = process_df(df2, 'mar')
df1 = process_df(df1, 'jun')
df4 = process_df(df4, 'nov')

In [ ]:

dfs = [df1, df2, df3, df4]
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = merged_df.merge(df, on='tile_id', how='outer')

In [ ]:
merged_df.head()

,MSAVI_p25_jun,NBR_p25_jun,NDMI_p25_jun,NDVI_p25_jun,NDWI_p25_jun,SAVI_p25_jun,tile_id,MSAVI_p25_mar,NBR_p25_mar,NDMI_p25_mar,...,NDMI_p25_aug,NDVI_p25_aug,NDWI_p25_aug,SAVI_p25_aug,MSAVI_p25_nov,NBR_p25_nov,NDMI_p25_nov,NDVI_p25_nov,NDWI_p25_nov,SAVI_p25_nov
0,0.050060,-0.214511,-0.344930,0.134944,-0.443714,0.062798,1,0.080544,-0.083369,-0.250438,...,-0.391111,0.121752,-0.459460,0.051674,-0.001906,-0.222408,-0.641935,-0.010937,-0.613095,-0.002632
1,0.052764,-0.212984,-0.337837,0.133041,-0.404882,0.064198,2,0.121340,0.014101,-0.210264,...,-0.373398,0.128352,-0.415888,0.053807,-0.002034,-0.228463,-0.620636,-0.013612,-0.596277,-0.002841
2,0.058036,-0.249858,-0.324795,0.129487,-0.375934,0.069275,3,0.109964,-0.039328,-0.168484,...,-0.322069,0.131976,-0.357738,0.065544,0.015465,-0.176001,-0.331988,0.053088,-0.600478,0.020151
3,0.048100,-0.266921,-0.317418,0.134012,-0.385697,0.062203,4,0.128730,-0.022126,-0.139722,...,-0.296555,0.142603,-0.354761,0.066433,0.007666,-0.195060,-0.334441,0.053169,-0.610037,0.009952
4,0.045236,-0.238468,-0.295708,0.141617,-0.391979,0.058252,5,0.148957,0.014118,-0.099199,...,-0.285542,0.147223,-0.358976,0.070579,0.023649,-0.147046,-0.427766,0.106109,-0.601346,0.030614


In [ ]:
merged_df.columns

Index(['MSAVI_p25_jun', 'NBR_p25_jun', 'NDMI_p25_jun', 'NDVI_p25_jun',
       'NDWI_p25_jun', 'SAVI_p25_jun', 'tile_id', 'MSAVI_p25_mar',
       'NBR_p25_mar', 'NDMI_p25_mar', 'NDVI_p25_mar', 'NDWI_p25_mar',
       'SAVI_p25_mar', 'MSAVI_p25_aug', 'NBR_p25_aug', 'NDMI_p25_aug',
       'NDVI_p25_aug', 'NDWI_p25_aug', 'SAVI_p25_aug', 'MSAVI_p25_nov',
       'NBR_p25_nov', 'NDMI_p25_nov', 'NDVI_p25_nov', 'NDWI_p25_nov',
       'SAVI_p25_nov'],
      dtype='object')

In [ ]:
merged_df.to_csv('/content/drive/MyDrive/GEE_Exports/train_data/Landsat_indices_surface_total_italy.csv')

## Modis evi

In [ ]:
modis_df = pd.read_csv('/content/drive/MyDrive/GEE_Exports/modis_evi_italy.csv')
modis_df.head()

,system:index,Id,Month,mean,tile_id,.geo
0,0_0,0,1.0,2968.383875,1,"{""type"":""Point"",""coordinates"":[17.881866249185..."
1,0_1,0,1.0,2968.493251,2,"{""type"":""Point"",""coordinates"":[17.882218988993..."
2,0_2,0,1.0,2968.602629,3,"{""type"":""Point"",""coordinates"":[17.882571728483..."
3,0_3,0,1.0,2968.821391,4,"{""type"":""Point"",""coordinates"":[17.882924467654..."
4,0_4,0,1.0,2968.930776,5,"{""type"":""Point"",""coordinates"":[17.883277206506..."


In [ ]:
modis_df.columns

Index(['system:index', 'Id', 'Month', 'mean', 'tile_id', '.geo'], dtype='object')

In [ ]:
modis_df["Month"] = modis_df["Month"].map(month_map)


In [ ]:
#rename column
modis_df.rename(columns={'mean': 'modis_evi'}, inplace=True)

In [ ]:
modis_df.head()

,system:index,Id,Month,modis_evi,tile_id,.geo
0,0_0,0,Jan,2968.383875,1,"{""type"":""Point"",""coordinates"":[17.881866249185..."
1,0_1,0,Jan,2968.493251,2,"{""type"":""Point"",""coordinates"":[17.882218988993..."
2,0_2,0,Jan,2968.602629,3,"{""type"":""Point"",""coordinates"":[17.882571728483..."
3,0_3,0,Jan,2968.821391,4,"{""type"":""Point"",""coordinates"":[17.882924467654..."
4,0_4,0,Jan,2968.930776,5,"{""type"":""Point"",""coordinates"":[17.883277206506..."


In [ ]:
modis_df.drop(['.geo', 'system:index', 'Id'], axis = 1, inplace = True)

In [ ]:
pivoted_df = modis_df.pivot(index='tile_id', columns='Month')
pivoted_df


modis_evi                                                      \
Month            Apr          Aug          Dec          Feb          Jan   
tile_id                                                                    
1        2998.016742  2052.074201  2985.623352  3089.916517  2968.383875   
2        2997.978277  2052.006226  2985.686682  3089.962582  2968.493251   
3        2997.939873  2051.938259  2985.750007  3090.008647  2968.602629   
4        2997.863254  2051.802352  2985.876639  3090.100783  2968.821391   
5        2997.825038  2051.734411  2985.939946  3090.146852  2968.930776   
...              ...          ...          ...          ...          ...   
27885    3005.527078  2044.966316  3002.638869  3128.173369  3024.989277   
27886    3005.526646  2044.915059  3002.701780  3128.243018  3025.120318   
27887    3005.526279  2044.863825  3002.764675  3128.312656  3025.251345   
27888    3005.525977  2044.812615  3002.827553  3128.382283  3025.382357   
27889    3005.525567  2044.710268  3002.953257  3128.521504  3025.644338   

                                                                          \
Month            Jul          Jun          Mar          May          Nov   
tile_id                                                                    
1        2100.010649  2158.683099  3215.151631  2359.272072  2514.216327   
2        2099.946585  2158.580680  3215.154296  2359.184164  2514.287793   
3        2099.882534  2158.478280  3215.156980  2359.096300  2514.359282   
4        2099.754473  2158.273537  3215.162403  2358.920702  2514.502324   
5        2099.690463  2158.171195  3215.165143  2358.832969  2514.573878   
...              ...          ...          ...          ...          ...   
27885    2092.811441  2139.227088  3243.890404  2342.086653  2554.297568   
27886    2092.765838  2139.145587  3243.920864  2342.029389  2554.391028   
27887    2092.720265  2139.064132  3243.951337  2341.972192  2554.484504   
27888    2092.674723  2138.982720  3243.981822  2341.915062  2554.577995   
27889    2092.583729  2138.820032  3244.042828  2341.801005  2554.765025   

                                   
Month            Oct          Sep  
tile_id                            
1        2531.766460  2165.138899  
2        2531.843376  2165.060730  
3        2531.920295  2164.982559  
4        2532.074143  2164.826212  
5        2532.151073  2164.748036  
...              ...          ...  
27885    2569.914857  2159.812603  
27886    2570.010843  2159.749271  
27887    2570.106830  2159.685952  
27888    2570.202818  2159.622646  
27889    2570.394795  2159.496074  

[27889 rows x 12 columns]

In [ ]:
pivoted_df.columns = [f"{col[0].lower()}_{col[1].lower()}" for col in pivoted_df.columns]
pivoted_df.reset_index(inplace=True)
pivoted_df

,tile_id,modis_evi_apr,modis_evi_aug,modis_evi_dec,modis_evi_feb,modis_evi_jan,modis_evi_jul,modis_evi_jun,modis_evi_mar,modis_evi_may,modis_evi_nov,modis_evi_oct,modis_evi_sep
0,1,2998.016742,2052.074201,2985.623352,3089.916517,2968.383875,2100.010649,2158.683099,3215.151631,2359.272072,2514.216327,2531.766460,2165.138899
1,2,2997.978277,2052.006226,2985.686682,3089.962582,2968.493251,2099.946585,2158.580680,3215.154296,2359.184164,2514.287793,2531.843376,2165.060730
2,3,2997.939873,2051.938259,2985.750007,3090.008647,2968.602629,2099.882534,2158.478280,3215.156980,2359.096300,2514.359282,2531.920295,2164.982559
3,4,2997.863254,2051.802352,2985.876639,3090.100783,2968.821391,2099.754473,2158.273537,3215.162403,2358.920702,2514.502324,2532.074143,2164.826212
4,5,2997.825038,2051.734411,2985.939946,3090.146852,2968.930776,2099.690463,2158.171195,3215.165143,2358.832969,2514.573878,2532.151073,2164.748036
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27884,27885,3005.527078,2044.966316,3002.638869,3128.173369,3024.989277,2092.811441,2139.227088,3243.890404,2342.086653,2554.297568,2569.914857,2159.812603
27885,27886,3005.526646,2044.915059,3002.701780,3128.243018,3025.120318,2092.765838,2139.145587,3243.920864,2342.029389,2554.391028,2570.010843,2159.749271
27886,27887,3005.526279,2044.863825,3002.764675,3128.312656,3025.251345,2092.720265,2139.064132,3243.951337,2341.972192,2554.484504,2570.106830,2159.685952
27887,27888,3005.525977,2044.812615,3002.827553,3128.382283,3025.382357,2092.674723,2138.982720,3243.981822,2341.915062,2554.577995,2570.202818,2159.622646


In [ ]:
pivoted_df.to_csv('/content/drive/MyDrive/GEE_Exports/train_data/modis_evi_italy.csv')

## modis daytime

In [ ]:
# /content/drive/MyDrive/GEE_Exports/modis_lst_daytime_italy.csv
# /content/drive/MyDrive/GEE_Exports/modis_lst_nightime_italy.csv

modis_daytime = pd.read_csv('/content/drive/MyDrive/GEE_Exports/modis_lst_daytime_italy.csv')
modis_daytime.head()

,system:index,Id,LST_Day_1km_p5,LST_Day_1km_p50,LST_Day_1km_p95,Month,tile_id,.geo
0,0_0,0,13991.221007,14248.996834,14415.196529,1.0,1,"{""type"":""Point"",""coordinates"":[17.881866249185..."
1,0_1,0,13991.234819,14249.006928,14415.207221,1.0,2,"{""type"":""Point"",""coordinates"":[17.882218988993..."
2,0_2,0,13991.248626,14249.017020,14415.217914,1.0,3,"{""type"":""Point"",""coordinates"":[17.882571728483..."
3,0_3,0,13991.276227,14249.037197,14415.239298,1.0,4,"{""type"":""Point"",""coordinates"":[17.882924467654..."
4,0_4,0,13991.290021,14249.047283,14415.249991,1.0,5,"{""type"":""Point"",""coordinates"":[17.883277206506..."


In [ ]:
modis_daytime.drop(['.geo', 'system:index', 'Id'], axis = 1, inplace = True)

In [ ]:
modis_daytime["Month"] = modis_daytime["Month"].map(month_map)


In [ ]:
pivoted_df = modis_daytime.pivot(index='tile_id', columns='Month')
pivoted_df


LST_Day_1km_p5                                            \
Month              Apr           Aug           Dec           Feb   
tile_id                                                            
1         14540.431071  15311.512142  14029.611194  14056.369026   
2         14540.426200  15311.513172  14029.624330  14056.386379   
3         14540.421324  15311.514200  14029.637462  14056.403729   
4         14540.411561  15311.516248  14029.663718  14056.438422   
5         14540.406672  15311.517268  14029.676841  14056.455766   
...                ...           ...           ...           ...   
27885     14541.204270  15313.900250  14032.962531  14061.025559   
27886     14541.197622  15313.900109  14032.975459  14061.042851   
27887     14541.190970  15313.899964  14032.988383  14061.060139   
27888     14541.184314  15313.899814  14033.001302  14061.077423   
27889     14541.170989  15313.899503  14033.027124  14061.111976   

                                                                               \
Month             Jan           Jul           Jun           Mar           May   
tile_id                                                                         
1        13991.221007  15358.432234  15137.738525  14259.260956  14826.644457   
2        13991.234819  15358.424177  15137.731132  14259.259116  14826.642919   
3        13991.248626  15358.416115  15137.723731  14259.257274  14826.641374   
4        13991.276227  15358.399976  15137.708907  14259.253586  14826.638267   
5        13991.290021  15358.391899  15137.701483  14259.251741  14826.636704   
...               ...           ...           ...           ...           ...   
27885    13994.736971  15358.874424  15138.670669  14259.793960  14828.274889   
27886    13994.750281  15358.864480  15138.660570  14259.791656  14828.271305   
27887    13994.763585  15358.854529  15138.650463  14259.789350  14828.267715   
27888    13994.776882  15358.844573  15138.640348  14259.787044  14828.264118   
27889    13994.803457  15358.824641  15138.620093  14259.782426  14828.256907   

                       ... LST_Day_1km_p95                              \
Month             Nov  ...             Dec           Feb           Jan   
tile_id                ...                                               
1        14214.310304  ...    14431.054992  14557.021903  14415.196529   
2        14214.312952  ...    14431.066240  14557.031328  14415.207221   
3        14214.315598  ...    14431.077489  14557.040754  14415.217914   
4        14214.320886  ...    14431.099986  14557.059611  14415.239298   
5        14214.323528  ...    14431.111234  14557.069042  14415.249991   
...               ...  ...             ...           ...           ...   
27885    14215.316420  ...    14433.572578  14559.607452  14417.795356   
27886    14215.319108  ...    14433.584283  14559.617398  14417.806492   
27887    14215.321794  ...    14433.595986  14559.627344  14417.817626   
27888    14215.324478  ...    14433.607687  14559.637290  14417.828758   
27889    14215.329840  ...    14433.631085  14559.657181  14417.851017   

                                                                               \
Month             Jul           Jun           Mar           May           Nov   
tile_id                                                                         
1        15930.279513  15849.945250  14877.065968  15564.030380  14751.537466   
2        15930.277211  15849.943201  14877.063335  15564.031203  14751.542419   
3        15930.274908  15849.941147  14877.060700  15564.032020  14751.547372   
4        15930.270294  15849.937022  14877.055428  15564.033633  14751.557279   
5        15930.267984  15849.934952  14877.052791  15564.034430  14751.562233   
...               ...           ...           ...           ...           ...   
27885    15932.547738  15852.587661  14877.904738  15567.885105  14753.072635   
27886    15932.543941  15852.583295  14877.901227  15567.882750  14753.07

In [ ]:
pivoted_df.columns = [f"{col[0].lower()}_{col[1].lower()}" for col in pivoted_df.columns]
pivoted_df.reset_index(inplace=True)
pivoted_df

,tile_id,lst_day_1km_p5_apr,lst_day_1km_p5_aug,lst_day_1km_p5_dec,lst_day_1km_p5_feb,lst_day_1km_p5_jan,lst_day_1km_p5_jul,lst_day_1km_p5_jun,lst_day_1km_p5_mar,lst_day_1km_p5_may,...,lst_day_1km_p95_dec,lst_day_1km_p95_feb,lst_day_1km_p95_jan,lst_day_1km_p95_jul,lst_day_1km_p95_jun,lst_day_1km_p95_mar,lst_day_1km_p95_may,lst_day_1km_p95_nov,lst_day_1km_p95_oct,lst_day_1km_p95_sep
0,1,14540.431071,15311.512142,14029.611194,14056.369026,13991.221007,15358.432234,15137.738525,14259.260956,14826.644457,...,14431.054992,14557.021903,14415.196529,15930.279513,15849.945250,14877.065968,15564.030380,14751.537466,15176.518486,15583.097641
1,2,14540.426200,15311.513172,14029.624330,14056.386379,13991.234819,15358.424177,15137.731132,14259.259116,14826.642919,...,14431.066240,14557.031328,14415.207221,15930.277211,15849.943201,14877.063335,15564.031203,14751.542419,15176.527393,15583.103345
2,3,14540.421324,15311.514200,14029.637462,14056.403729,13991.248626,15358.416115,15137.723731,14259.257274,14826.641374,...,14431.077489,14557.040754,14415.217914,15930.274908,15849.941147,14877.060700,15564.032020,14751.547372,15176.536304,15583.109050
3,4,14540.411561,15311.516248,14029.663718,14056.438422,13991.276227,15358.399976,15137.708907,14259.253586,14826.638267,...,14431.099986,14557.059611,14415.239298,15930.270294,15849.937022,14877.055428,15564.033633,14751.557279,15176.554136,15583.120465
4,5,14540.406672,15311.517268,14029.676841,14056.455766,13991.290021,15358.391899,15137.701483,14259.251741,14826.636704,...,14431.111234,14557.069042,14415.249991,15930.267984,15849.934952,14877.052791,15564.034430,14751.562233,15176.563056,15583.126174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27884,27885,14541.204270,15313.900250,14032.962531,14061.025559,13994.736971,15358.874424,15138.670669,14259.793960,14828.274889,...,14433.572578,14559.607452,14417.795356,15932.547738,15852.587661,14877.904738,15567.885105,14753.072635,15179.925737,15586.265353
27885,27886,14541.197622,15313.900109,14032.975459,14061.042851,13994.750281,15358.864480,15138.660570,14259.791656,14828.271305,...,14433.584283,14559.617398,14417.806492,15932.543941,15852.583295,14877.901227,15567.882750,14753.077844,15179.935161,15586.270764
27886,27887,14541.190970,15313.899964,14032.988383,14061.060139,13994.763585,15358.854529,15138.650463,14259.789350,14828.267715,...,14433.595986,14559.627344,14417.817626,15932.540140,15852.578920,14877.897714,15567.880386,14753.083051,15179.944585,15586.276174
27887,27888,14541.184314,15313.899814,14033.001302,14061.077423,13994.776882,15358.844573,15138.640348,14259.787044,14828.264118,...,14433.607687,14559.637290,14417.828758,15932.536334,15852.574538,14877.894200,15567.878012,14753.088258,15179.954010,15586.281583


In [ ]:
pivoted_df.to_csv('/content/drive/MyDrive/GEE_Exports/train_data/modis_lst_daytime_italy.csv')

## Modis Lst Nightime

In [ ]:
modis_lst_nightime = pd.read_csv('/content/drive/MyDrive/GEE_Exports/modis_lst_nightime_italy.csv')
modis_lst_nightime.head()

,system:index,Id,LST_Night_1km_p5,LST_Night_1km_p50,LST_Night_1km_p95,Month,tile_id,.geo
0,0_0,0,13650.461566,13870.473182,14105.511257,1.0,1,"{""type"":""Point"",""coordinates"":[17.881866249185..."
1,0_1,0,13650.481294,13870.493908,14105.533508,1.0,2,"{""type"":""Point"",""coordinates"":[17.882218988993..."
2,0_2,0,13650.501019,13870.514631,14105.555755,1.0,3,"{""type"":""Point"",""coordinates"":[17.882571728483..."
3,0_3,0,13650.540459,13870.556065,14105.600235,1.0,4,"{""type"":""Point"",""coordinates"":[17.882924467654..."
4,0_4,0,13650.560174,13870.576777,14105.622469,1.0,5,"{""type"":""Point"",""coordinates"":[17.883277206506..."


In [ ]:
modis_lst_nightime.shape

(334668, 8)

In [ ]:
modis_lst_nightime.drop(['.geo', 'system:index', 'Id'], axis = 1, inplace = True)

In [ ]:
modis_lst_nightime["Month"] = modis_lst_nightime["Month"].map(month_map)


In [ ]:
pivoted_df = modis_lst_nightime.pivot(index='tile_id', columns='Month')
pivoted_df


LST_Night_1km_p5                                            \
Month                Apr           Aug           Dec           Feb   
tile_id                                                              
1           13923.396098  14567.707397  13693.505042  13676.722717   
2           13923.410969  14567.725335  13693.520015  13676.744477   
3           13923.425837  14567.743271  13693.534985  13676.766234   
4           13923.455563  14567.779135  13693.564918  13676.809741   
5           13923.470421  14567.797065  13693.579880  13676.831491   
...                  ...           ...           ...           ...   
27885       13926.337662  14571.187763  13696.015692  13681.207653   
27886       13926.352460  14571.205826  13696.031070  13681.229633   
27887       13926.367253  14571.223886  13696.046443  13681.251607   
27888       13926.382042  14571.241941  13696.061811  13681.273576   
27889       13926.411608  14571.278041  13696.092534  13681.317498   

                                                                               \
Month             Jan           Jul           Jun           Mar           May   
tile_id                                                                         
1        13650.461566  14569.785435  14390.469359  13750.944564  14161.290858   
2        13650.481294  14569.798581  14390.479771  13750.963426  14161.300790   
3        13650.501019  14569.811722  14390.490179  13750.982284  14161.310719   
4        13650.540459  14569.837993  14390.510980  13751.019989  14161.330568   
5        13650.560174  14569.851122  14390.521374  13751.038836  14161.340487   
...               ...           ...           ...           ...           ...   
27885    13654.349519  14572.261227  14392.534790  13754.536231  14163.201311   
27886    13654.369439  14572.273952  14392.544542  13754.555091  14163.210989   
27887    13654.389354  14572.286672  14392.554288  13754.573945  14163.220664   
27888    13654.409262  14572.299386  14392.564029  13754.592795  14163.230336   
27889    13654.449059  14572.324801  14392.583494  13754.630477  14163.249670   

                       ... LST_Night_1km_p95                              \
Month             Nov  ...               Dec           Feb           Jan   
tile_id                ...                                                 
1        13871.459964  ...      14168.829486  14098.075268  14105.511257   
2        13871.484244  ...      14168.846445  14098.100063  14105.533508   
3        13871.508521  ...      14168.863400  14098.124857  14105.555755   
4        13871.557063  ...      14168.897295  14098.174439  14105.600235   
5        13871.581328  ...      14168.914236  14098.199227  14105.622469   
...               ...  ...               ...           ...           ...   
27885    13874.908652  ...      14172.983408  14103.600598  14110.801341   
27886    13874.932058  ...      14172.999685  14103.625520  14110.823061   
27887    13874.955457  ...      14173.015954  14103.650438  14110.844775   
27888    13874.978850  ...      14173.032216  14103.675351  14110.866482   
27889    13875.025618  ...      14173.064719  14103.725166  14110.909874   

                                                                               \
Month             Jul           Jun           Mar           May           Nov   
tile_id                                                                         
1        14923.842534  14835.203821  14183.092107  14568.140801  14343.502880   
2        14923.847921  14835.206915  14183.106716  14568.148915  14343.533483   
3        14923.853305  14835.210005  14183.121323  14568.157025  14343.564085   
4        14923.864059  14835.216174  14183.150531  14568.173236  14343.625287   
5        14923.869431  14835.219253  14183.165133  14568.181337  14343.655886   
...               ...           ...           ...           ...           ...   
27885    14924.359031  14835.379309  14186.136405  14569.682571  14353.191299   
27886    14924.36

In [ ]:
pivoted_df.columns = [f"{col[0].lower()}_{col[1].lower()}" for col in pivoted_df.columns]
pivoted_df.reset_index(inplace=True)
pivoted_df

,tile_id,lst_night_1km_p5_apr,lst_night_1km_p5_aug,lst_night_1km_p5_dec,lst_night_1km_p5_feb,lst_night_1km_p5_jan,lst_night_1km_p5_jul,lst_night_1km_p5_jun,lst_night_1km_p5_mar,lst_night_1km_p5_may,...,lst_night_1km_p95_dec,lst_night_1km_p95_feb,lst_night_1km_p95_jan,lst_night_1km_p95_jul,lst_night_1km_p95_jun,lst_night_1km_p95_mar,lst_night_1km_p95_may,lst_night_1km_p95_nov,lst_night_1km_p95_oct,lst_night_1km_p95_sep
0,1,13923.396098,14567.707397,13693.505042,13676.722717,13650.461566,14569.785435,14390.469359,13750.944564,14161.290858,...,14168.829486,14098.075268,14105.511257,14923.842534,14835.203821,14183.092107,14568.140801,14343.502880,14533.366213,14752.579923
1,2,13923.410969,14567.725335,13693.520015,13676.744477,13650.481294,14569.798581,14390.479771,13750.963426,14161.300790,...,14168.846445,14098.100063,14105.533508,14923.847921,14835.206915,14183.106716,14568.148915,14343.533483,14533.379451,14752.591050
2,3,13923.425837,14567.743271,13693.534985,13676.766234,13650.501019,14569.811722,14390.490179,13750.982284,14161.310719,...,14168.863400,14098.124857,14105.555755,14923.853305,14835.210005,14183.121323,14568.157025,14343.564085,14533.392685,14752.602174
3,4,13923.455563,14567.779135,13693.564918,13676.809741,13650.540459,14569.837993,14390.510980,13751.019989,14161.330568,...,14168.897295,14098.174439,14105.600235,14923.864059,14835.216174,14183.150531,14568.173236,14343.625287,14533.419146,14752.624410
4,5,13923.470421,14567.797065,13693.579880,13676.831491,13650.560174,14569.851122,14390.521374,13751.038836,14161.340487,...,14168.914236,14098.199227,14105.622469,14923.869431,14835.219253,14183.165133,14568.181337,14343.655886,14533.432371,14752.635523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27884,27885,13926.337662,14571.187763,13696.015692,13681.207653,13654.349519,14572.261227,14392.534790,13754.536231,14163.201311,...,14172.983408,14103.600598,14110.801341,14924.359031,14835.379309,14186.136405,14569.682571,14353.191299,14536.168219,14754.584440
27885,27886,13926.352460,14571.205826,13696.031070,13681.229633,13654.369439,14572.273952,14392.544542,13754.555091,14163.210989,...,14172.999685,14103.625520,14110.823061,14924.363986,14835.381877,14186.151026,14569.690215,14353.224356,14536.181219,14754.595219
27886,27887,13926.367253,14571.223886,13696.046443,13681.251607,13654.389354,14572.286672,14392.554288,13754.573945,14163.220664,...,14173.015954,14103.650438,14110.844775,14924.368937,14835.384441,14186.165643,14569.697855,14353.257408,14536.194214,14754.605994
27887,27888,13926.382042,14571.241941,13696.061811,13681.273576,13654.409262,14572.299386,14392.564029,13754.592795,14163.230336,...,14173.032216,14103.675351,14110.866482,14924.373883,14835.387001,14186.180257,14569.705491,14353.290456,14536.207205,14754.616764


In [ ]:
pivoted_df.to_csv('/content/drive/MyDrive/GEE_Exports/train_data/modis_lst_nightime_italy.csv')

In [ ]:

# /content/drive/MyDrive/GEE_Exports/MERIT_height_above_nearest_neighbour_italy.csv
# /content/drive/MyDrive/GEE_Exports/Monthly_precipitation_italy.csv
# /content/drive/MyDrive/GEE_Exports/elevation_ITALY.csvv
# /content/drive/MyDrive/GEE_Exports/soil_organic_carbon_250m_ITALY.csv

df1 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/MERIT_UPSTREAM_AREA_ITALY.csv')
df1.head()

,system:index,Id,mean,tile_id,.geo
0,0,0,0.045670,1,"{""type"":""Point"",""coordinates"":[17.881866249185..."
1,1,0,0.117439,2,"{""type"":""Point"",""coordinates"":[17.882218988993..."
2,2,0,0.117439,3,"{""type"":""Point"",""coordinates"":[17.882571728483..."
3,3,0,0.117439,4,"{""type"":""Point"",""coordinates"":[17.882924467654..."
4,4,0,0.065243,5,"{""type"":""Point"",""coordinates"":[17.883277206506..."


In [ ]:
df1.rename(columns={'mean': 'upstream'}, inplace=True)

In [ ]:
df1.drop(['.geo', 'system:index', 'Id'], axis = 1, inplace = True)

In [ ]:
df1.to_csv('/content/drive/MyDrive/GEE_Exports/train_data/MERIT_UPSTREAM_AREA_ITALY.csv')

In [ ]:
# /content/drive/MyDrive/GEE_Exports/MERIT_height_above_nearest_neighbour_italy.csv
df2 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/MERIT_height_above_nearest_neighbour_italy.csv')
df2.head()

,system:index,Id,mean,tile_id,.geo
0,0,0,4.200005,1,"{""type"":""Point"",""coordinates"":[17.881866249185..."
1,1,0,4.200005,2,"{""type"":""Point"",""coordinates"":[17.882218988993..."
2,2,0,4.200005,3,"{""type"":""Point"",""coordinates"":[17.882571728483..."
3,3,0,4.200005,4,"{""type"":""Point"",""coordinates"":[17.882924467654..."
4,4,0,4.200005,5,"{""type"":""Point"",""coordinates"":[17.883277206506..."


In [ ]:
df2.rename(columns={'mean': 'hand'}, inplace=True)

In [ ]:
df2.drop(['.geo', 'system:index', 'Id'], axis = 1, inplace = True)

In [ ]:
df2.to_csv('/content/drive/MyDrive/GEE_Exports/train_data/MERIT_height_above_nearest_neighbour_italy.csv')

In [ ]:
# /content/drive/MyDrive/GEE_Exports/Monthly_precipitation_italy.csv
# /content/drive/MyDrive/GEE_Exports/elevation_ITALY.csvv
# /content/drive/MyDrive/GEE_Exports/soil_organic_carbon_250m_ITALY.csv
df3 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/Monthly_precipitation_italy.csv')
df3.head()

,system:index,Id,Month,mean,tile_id,.geo
0,0_0,0,1.0,63.0,1,"{""type"":""Point"",""coordinates"":[17.881866249185..."
1,0_1,0,1.0,63.0,2,"{""type"":""Point"",""coordinates"":[17.882218988993..."
2,0_2,0,1.0,63.0,3,"{""type"":""Point"",""coordinates"":[17.882571728483..."
3,0_3,0,1.0,63.0,4,"{""type"":""Point"",""coordinates"":[17.882924467654..."
4,0_4,0,1.0,63.0,5,"{""type"":""Point"",""coordinates"":[17.883277206506..."


In [ ]:
df3.rename(columns={'mean': 'precipitation'}, inplace=True)

In [ ]:
df3.drop(['.geo', 'system:index', 'Id'], axis = 1, inplace = True)

In [ ]:
df3['Month']= df3['Month'].map(month_map)

In [ ]:
df3.shape

(334668, 3)

In [ ]:
pivoted_df = df3.pivot(index='tile_id', columns='Month')
pivoted_df


precipitation                                                        \
Month             Apr   Aug   Dec   Feb   Jan   Jul   Jun   Mar   May   Nov   
tile_id                                                                       
1                38.0  26.0  69.0  59.0  63.0  15.0  21.0  67.0  31.0  78.0   
2                38.0  26.0  69.0  59.0  63.0  15.0  21.0  67.0  31.0  78.0   
3                38.0  26.0  69.0  59.0  63.0  15.0  21.0  67.0  31.0  78.0   
4                38.0  26.0  69.0  59.0  63.0  15.0  21.0  67.0  31.0  78.0   
5                38.0  26.0  69.0  59.0  63.0  15.0  21.0  67.0  31.0  78.0   
...               ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
27885            38.0  26.0  69.0  61.0  65.0  14.0  20.0  68.0  31.0  79.0   
27886            38.0  26.0  69.0  61.0  65.0  14.0  20.0  68.0  31.0  79.0   
27887            38.0  26.0  69.0  61.0  65.0  14.0  20.0  68.0  31.0  79.0   
27888            38.0  26.0  69.0  61.0  65.0  14.0  20.0  68.0  31.0  79.0   
27889            38.0  26.0  69.0  61.0  65.0  14.0  20.0  68.0  31.0  79.0   

                     
Month     Oct   Sep  
tile_id              
1        72.0  47.0  
2        72.0  47.0  
3        72.0  47.0  
4        72.0  47.0  
5        72.0  47.0  
...       ...   ...  
27885    73.0  48.0  
27886    73.0  48.0  
27887    73.0  48.0  
27888    73.0  48.0  
27889    73.0  48.0  

[27889 rows x 12 columns]

In [ ]:
pivoted_df.columns = [f"{col[0].lower()}_{col[1].lower()}" for col in pivoted_df.columns]
pivoted_df.reset_index(inplace=True)
pivoted_df

,tile_id,precipitation_apr,precipitation_aug,precipitation_dec,precipitation_feb,precipitation_jan,precipitation_jul,precipitation_jun,precipitation_mar,precipitation_may,precipitation_nov,precipitation_oct,precipitation_sep
0,1,38.0,26.0,69.0,59.0,63.0,15.0,21.0,67.0,31.0,78.0,72.0,47.0
1,2,38.0,26.0,69.0,59.0,63.0,15.0,21.0,67.0,31.0,78.0,72.0,47.0
2,3,38.0,26.0,69.0,59.0,63.0,15.0,21.0,67.0,31.0,78.0,72.0,47.0
3,4,38.0,26.0,69.0,59.0,63.0,15.0,21.0,67.0,31.0,78.0,72.0,47.0
4,5,38.0,26.0,69.0,59.0,63.0,15.0,21.0,67.0,31.0,78.0,72.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27884,27885,38.0,26.0,69.0,61.0,65.0,14.0,20.0,68.0,31.0,79.0,73.0,48.0
27885,27886,38.0,26.0,69.0,61.0,65.0,14.0,20.0,68.0,31.0,79.0,73.0,48.0
27886,27887,38.0,26.0,69.0,61.0,65.0,14.0,20.0,68.0,31.0,79.0,73.0,48.0
27887,27888,38.0,26.0,69.0,61.0,65.0,14.0,20.0,68.0,31.0,79.0,73.0,48.0


In [ ]:
pivoted_df.to_csv('/content/drive/MyDrive/GEE_Exports/train_data/Monthly_precipitation_italy.csv')

In [ ]:

# /content/drive/MyDrive/GEE_Exports/elevation_ITALY.csvv
# /content/drive/MyDrive/GEE_Exports/soil_organic_carbon_250m_ITALY.csv
df4 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/elevation_ITALY.csv')
df4.head()

,system:index,Id,mean,tile_id,.geo
0,0,0,63.0,1,"{""type"":""Point"",""coordinates"":[17.881866249185..."
1,1,0,63.0,2,"{""type"":""Point"",""coordinates"":[17.882218988993..."
2,2,0,64.0,3,"{""type"":""Point"",""coordinates"":[17.882571728483..."
3,3,0,64.0,4,"{""type"":""Point"",""coordinates"":[17.882924467654..."
4,4,0,64.0,5,"{""type"":""Point"",""coordinates"":[17.883277206506..."


In [ ]:
df4.rename(columns={'mean': 'elevation'}, inplace=True)

In [ ]:
df4.drop(['.geo', 'system:index', 'Id'], axis = 1, inplace = True)

In [ ]:
df4.shape

(27889, 2)

In [ ]:
df4.to_csv('/content/drive/MyDrive/GEE_Exports/train_data/elevation_ITALY.csv')

In [ ]:
# /content/drive/MyDrive/GEE_Exports/soil_organic_carbon_250m_ITALY.csv
df5 = pd.read_csv('/content/drive/MyDrive/GEE_Exports/soil_organic_carbon_250m_ITALY.csv')
df5.head()

,system:index,Id,b0,b10,b30,b60,tile_id,.geo
0,0,0,2.0,2.0,2.0,1.0,1,"{""type"":""Point"",""coordinates"":[17.881866249185..."
1,1,0,2.0,2.0,2.0,1.0,2,"{""type"":""Point"",""coordinates"":[17.882218988993..."
2,2,0,2.0,2.0,2.0,1.0,3,"{""type"":""Point"",""coordinates"":[17.882571728483..."
3,3,0,2.0,2.0,2.0,1.0,4,"{""type"":""Point"",""coordinates"":[17.882924467654..."
4,4,0,2.0,2.0,2.0,1.0,5,"{""type"":""Point"",""coordinates"":[17.883277206506..."


In [ ]:
df5.drop(['.geo', 'system:index', 'Id'], axis = 1, inplace = True)

In [ ]:
df5.shape

(27889, 5)

In [ ]:
df5.to_csv('/content/drive/MyDrive/GEE_Exports/train_data/soil_organic_carbon_250m_ITALY.csv')

In [ ]:
path = '/content/drive/MyDrive/GEE_Exports/train_data'
csv_files = glob.glob("/content/drive/MyDrive/GEE_Exports/train_data/*.csv")
len(csv_files)

13

In [ ]:
#merge all the files in csv files
df1 = pd.read_csv(csv_files[0])
df2 = pd.read_csv(csv_files[1])
df3 = pd.read_csv(csv_files[2])
df4 = pd.read_csv(csv_files[3])
df5 = pd.read_csv(csv_files[4])
df6 = pd.read_csv(csv_files[5])
df7 = pd.read_csv(csv_files[6])
df8 = pd.read_csv(csv_files[7])
df9 = pd.read_csv(csv_files[8])
df10 = pd.read_csv(csv_files[9])
df11 = pd.read_csv(csv_files[10])
df12 = pd.read_csv(csv_files[11])
df13 = pd.read_csv(csv_files[12])


In [ ]:
def drop_unnamed(df):
  if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)
  return df


In [ ]:
df1 = drop_unnamed(df1)
df2 = drop_unnamed(df2)
df3 = drop_unnamed(df3)
df4 = drop_unnamed(df4)
df5 = drop_unnamed(df5)
df6 = drop_unnamed(df6)
df7 = drop_unnamed(df7)
df8 = drop_unnamed(df8)
df9 = drop_unnamed(df9)
df10 = drop_unnamed(df10)
df11 = drop_unnamed(df11)
df12 = drop_unnamed(df12)
df13 = drop_unnamed(df13)

In [ ]:
df4.columns

Index(['B_p25_aug', 'G_p25_aug', 'NIR_p25_aug', 'R_p25_aug', 'SWIR1_p25_aug',
       'SWIR2_p25_aug', 'tile_id', 'B_p25_nov', 'G_p25_nov', 'NIR_p25_nov',
       'R_p25_nov', 'SWIR1_p25_nov', 'SWIR2_p25_nov', 'B_p25_jun', 'G_p25_jun',
       'NIR_p25_jun', 'R_p25_jun', 'SWIR1_p25_jun', 'SWIR2_p25_jun',
       'B_p25_mar', 'G_p25_mar', 'NIR_p25_mar', 'R_p25_mar', 'SWIR1_p25_mar',
       'SWIR2_p25_mar'],
      dtype='object')

In [ ]:
#merge all the dfs on tile_id
dfs = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13]
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = merged_df.merge(df, on='tile_id', how='outer')


In [ ]:
merged_df

,B_landsat2000,G_landsat2000,Id_landsat2000,NIR_landsat2000,R_landsat2000,SWIR1_landsat2000,SWIR2_landsat2000,pixel_qa_landsat2000,tile_id,B_landsat2014,...,precipitation_mar,precipitation_may,precipitation_nov,precipitation_oct,precipitation_sep,elevation,b0,b10,b30,b60
0,0.027432,0.040278,0,0.092256,0.078286,0.243200,0.147339,22082.0,1,0.027432,...,67.0,31.0,78.0,72.0,47.0,63.0,2.0,2.0,2.0,1.0
1,0.031327,0.043320,0,0.095580,0.082576,0.257514,0.158139,22082.0,2,0.031327,...,67.0,31.0,78.0,72.0,47.0,63.0,2.0,2.0,2.0,1.0
2,0.038459,0.051663,0,0.099826,0.091118,0.310984,0.163467,22082.0,3,0.038459,...,67.0,31.0,78.0,72.0,47.0,64.0,2.0,2.0,2.0,1.0
3,0.037720,0.048854,0,0.098279,0.090716,0.322445,0.156067,22082.0,4,0.037720,...,67.0,31.0,78.0,72.0,47.0,64.0,2.0,2.0,2.0,1.0
4,0.035727,0.050009,0,0.104645,0.093037,0.310300,0.166551,22082.0,5,0.035727,...,67.0,31.0,78.0,72.0,47.0,64.0,2.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27884,0.042932,0.057097,0,0.115239,0.099451,0.241347,0.148324,21824.0,27885,0.042932,...,68.0,31.0,79.0,73.0,48.0,37.0,2.0,2.0,2.0,1.0
27885,0.042932,0.057097,0,0.115239,0.099451,0.241347,0.148324,21824.0,27886,0.042932,...,68.0,31.0,79.0,73.0,48.0,37.0,2.0,2.0,2.0,1.0
27886,0.041768,0.055616,0,0.113307,0.097150,0.234595,0.147206,21824.0,27887,0.041768,...,68.0,31.0,79.0,73.0,48.0,37.0,2.0,2.0,2.0,1.0
27887,0.081259,0.094775,0,0.151145,0.134576,0.265867,0.168866,25120.6,27888,0.081259,...,68.0,31.0,79.0,73.0,48.0,37.0,2.0,2.0,2.0,1.0


In [ ]:
merged_df.to_csv('/content/drive/MyDrive/GEE_Exports/train_data/train_data_italy.csv')

In [ ]:
merged_df.shape

(27889, 344)

In [ ]:
#columns to remove =
#'Id_landsat2024',
#'pixel_qa_landsat2000', 'tile_id',
#'MSAVI_p25_aug',
# 'NBR_p25_aug', 'NDMI_p25_aug', 'NDVI_p25_aug', 'NDWI_p25_aug',
# 'SAVI_p25_aug', 'MSAVI_p25_nov', 'NBR_p25_nov', 'NDMI_p25_nov',
# 'NDVI_p25_nov', 'NDWI_p25_nov', 'SAVI_p25_nov',
#rename this
#'b0', 'b10', 'b30', 'b60' to soc_b10 etc
#'upstream',
#'hand', add merit so it becomes merit_hand, and merit the other one, since the foolish letter no dey
#add monthly as a prefix to precipitation\



In [ ]:
import pandas as pd
df = pd.read_csv('train_data_italy.csv')
df.columns.values

array(['Unnamed: 0', 'B_landsat2000', 'G_landsat2000', 'Id_landsat2000',
       'NIR_landsat2000', 'R_landsat2000', 'SWIR1_landsat2000',
       'SWIR2_landsat2000', 'pixel_qa_landsat2000', 'tile_id',
       'B_landsat2014', 'G_landsat2014', 'Id_landsat2014',
       'NIR_landsat2014', 'R_landsat2014', 'SWIR1_landsat2014',
       'SWIR2_landsat2014', 'pixel_qa_landsat2014', 'B_landsat2024',
       'G_landsat2024', 'Id_landsat2024', 'NIR_landsat2024',
       'R_landsat2024', 'SWIR1_landsat2024', 'SWIR2_landsat2024',
       'pixel_qa_landsat2024', 'b_p25_apr', 'b_p25_aug', 'b_p25_dec',
       'b_p25_feb', 'b_p25_jan', 'b_p25_jul', 'b_p25_jun', 'b_p25_mar',
       'b_p25_may', 'b_p25_nov', 'b_p25_oct', 'b_p25_sep', 'evi_p25_apr',
       'evi_p25_aug', 'evi_p25_dec', 'evi_p25_feb', 'evi_p25_jan',
       'evi_p25_jul', 'evi_p25_jun', 'evi_p25_mar', 'evi_p25_may',
       'evi_p25_nov', 'evi_p25_oct', 'evi_p25_sep', 'g_p25_apr',
       'g_p25_aug', 'g_p25_dec', 'g_p25_feb', 'g_p25_jan', 'g_p25_

In [ ]:
df.shape

(27889, 345)

In [ ]:
df.drop(['Id_landsat2024', 'Unnamed: 0','pixel_qa_landsat2000', 'tile_id','MSAVI_p25_aug',
      'NBR_p25_aug', 'NDMI_p25_aug', 'NDVI_p25_aug', 'NDWI_p25_aug',
      'SAVI_p25_aug', 'MSAVI_p25_nov', 'NBR_p25_nov', 'NDMI_p25_nov',
      'NDVI_p25_nov', 'NDWI_p25_nov', 'SAVI_p25_nov', 'evi_p25_apr',
       'evi_p25_aug', 'evi_p25_dec', 'evi_p25_feb', 'evi_p25_jan',
       'evi_p25_jul', 'evi_p25_jun', 'evi_p25_mar', 'evi_p25_may',
       'evi_p25_nov', 'evi_p25_oct', 'evi_p25_sep',],axis = 1, inplace = True)



In [ ]:
# df.drop(['evi_p25_apr',
#        'evi_p25_aug', 'evi_p25_dec', 'evi_p25_feb', 'evi_p25_jan',
#        'evi_p25_jul', 'evi_p25_jun', 'evi_p25_mar', 'evi_p25_may',
#        'evi_p25_nov', 'evi_p25_oct', 'evi_p25_sep',], axis = 1, inplace = True)

In [ ]:
#rename columns
df.rename(columns={'b0': 'soc_b0', 'b10': 'soc_b10',
                   'b30': 'soc_b30', 'b60': 'soc_b60',
                   'elevation': "srtm", 'upstream': 'merit_upstream',
                   'hand': 'merit_hand',
                    'precipitation_apr': 'monthly_precipitation_apr',
                   'precipitation_aug': 'monthly_precipitation_aug',
                    'precipitation_dec': 'monthly_precipitation_dec',
                   'precipitation_feb': 'monthly_precipitation_feb',
                   'precipitation_jan': 'monthly_precipitation_jan',
                   'precipitation_jul': 'monthly_precipitation_jul',
                   'precipitation_jun': 'monthly_precipitation_jun',
                   'precipitation_mar': 'monthly_precipitation_mar',
                   'precipitation_may': 'monthly_precipitation_may',
                   'precipitation_nov': 'monthly_precipitation_nov',
                   'precipitation_oct': 'monthly_precipitation_oct',
                   'precipitation_sep': 'monthly_precipitation_sep'
                   }, inplace=True)



# 'precipitation_apr', 'precipitation_aug',
#        'precipitation_dec', 'precipitation_feb', 'precipitation_jan',
#        'precipitation_jul', 'precipitation_jun', 'precipitation_mar',
#        'precipitation_may', 'precipitation_nov', 'precipitation_oct',
#        'precipitation_sep'

In [ ]:
df.shape

(27889, 317)

In [ ]:
df.columns.values

array(['B_landsat2000', 'G_landsat2000', 'Id_landsat2000',
       'NIR_landsat2000', 'R_landsat2000', 'SWIR1_landsat2000',
       'SWIR2_landsat2000', 'B_landsat2014', 'G_landsat2014',
       'Id_landsat2014', 'NIR_landsat2014', 'R_landsat2014',
       'SWIR1_landsat2014', 'SWIR2_landsat2014', 'pixel_qa_landsat2014',
       'B_landsat2024', 'G_landsat2024', 'NIR_landsat2024',
       'R_landsat2024', 'SWIR1_landsat2024', 'SWIR2_landsat2024',
       'pixel_qa_landsat2024', 'b_p25_apr', 'b_p25_aug', 'b_p25_dec',
       'b_p25_feb', 'b_p25_jan', 'b_p25_jul', 'b_p25_jun', 'b_p25_mar',
       'b_p25_may', 'b_p25_nov', 'b_p25_oct', 'b_p25_sep', 'evi_p25_apr',
       'evi_p25_aug', 'evi_p25_dec', 'evi_p25_feb', 'evi_p25_jan',
       'evi_p25_jul', 'evi_p25_jun', 'evi_p25_mar', 'evi_p25_may',
       'evi_p25_nov', 'evi_p25_oct', 'evi_p25_sep', 'g_p25_apr',
       'g_p25_aug', 'g_p25_dec', 'g_p25_feb', 'g_p25_jan', 'g_p25_jul',
       'g_p25_jun', 'g_p25_mar', 'g_p25_may', 'g_p25_nov', 'g_p25_oc

In [ ]:
import pandas as pd
df2 = pd.read_csv('train_new_data_lucas.csv')
df2.columns.values

array(['B_p25_aug', 'G_p25_aug', 'NIR_p25_aug', 'POINTID', 'R_p25_aug',
       'SWIR1_p25_aug', 'SWIR2_p25_aug', 'B_p25_jun', 'G_p25_jun',
       'NIR_p25_jun', 'R_p25_jun', 'SWIR1_p25_jun', 'SWIR2_p25_jun',
       'B_p25_mar', 'G_p25_mar', 'NIR_p25_mar', 'R_p25_mar',
       'SWIR1_p25_mar', 'SWIR2_p25_mar', 'B_p25_nov', 'G_p25_nov',
       'NIR_p25_nov', 'R_p25_nov', 'SWIR1_p25_nov', 'SWIR2_p25_nov',
       'MSAVI_p25_jun', 'NBR_p25_jun', 'NDMI_p25_jun', 'NDVI_p25_jun',
       'NDWI_p25_jun', 'SAVI_p25_jun', 'MSAVI_p25_mar', 'NBR_p25_mar',
       'NDMI_p25_mar', 'NDVI_p25_mar', 'NDWI_p25_mar', 'SAVI_p25_mar',
       'B_landsat2000', 'G_landsat2000', 'NIR_landsat2000',
       'R_landsat2000', 'SWIR1_landsat2000', 'SWIR2_landsat2000',
       'B_landsat2014', 'G_landsat2014', 'NIR_landsat2014',
       'R_landsat2014', 'SWIR1_landsat2014', 'SWIR2_landsat2014',
       'B_landsat2024', 'G_landsat2024', 'NIR_landsat2024',
       'R_landsat2024', 'SWIR1_landsat2024', 'SWIR2_landsat2024',
    

In [ ]:
df2.shape

(16231, 317)

In [ ]:
list1 = ['B_p25_aug', 'G_p25_aug', 'NIR_p25_aug', 'POINTID', 'R_p25_aug',
       'SWIR1_p25_aug', 'SWIR2_p25_aug', 'B_p25_jun', 'G_p25_jun',
       'NIR_p25_jun', 'R_p25_jun', 'SWIR1_p25_jun', 'SWIR2_p25_jun',
       'B_p25_mar', 'G_p25_mar', 'NIR_p25_mar', 'R_p25_mar',
       'SWIR1_p25_mar', 'SWIR2_p25_mar', 'B_p25_nov', 'G_p25_nov',
       'NIR_p25_nov', 'R_p25_nov', 'SWIR1_p25_nov', 'SWIR2_p25_nov',
       'MSAVI_p25_jun', 'NBR_p25_jun', 'NDMI_p25_jun', 'NDVI_p25_jun',
       'NDWI_p25_jun', 'SAVI_p25_jun', 'MSAVI_p25_mar', 'NBR_p25_mar',
       'NDMI_p25_mar', 'NDVI_p25_mar', 'NDWI_p25_mar', 'SAVI_p25_mar',
       'B_landsat2000', 'G_landsat2000', 'NIR_landsat2000',
       'R_landsat2000', 'SWIR1_landsat2000', 'SWIR2_landsat2000',
       'B_landsat2014', 'G_landsat2014', 'NIR_landsat2014',
       'R_landsat2014', 'SWIR1_landsat2014', 'SWIR2_landsat2014',
       'B_landsat2024', 'G_landsat2024', 'NIR_landsat2024',
       'R_landsat2024', 'SWIR1_landsat2024', 'SWIR2_landsat2024',
       'b_p25_apr', 'b_p25_aug', 'b_p25_dec', 'b_p25_feb', 'b_p25_jan',
       'b_p25_jul', 'b_p25_jun', 'b_p25_mar', 'b_p25_may', 'b_p25_nov',
       'b_p25_oct', 'b_p25_sep', 'g_p25_apr', 'g_p25_aug', 'g_p25_dec',
       'g_p25_feb', 'g_p25_jan', 'g_p25_jul', 'g_p25_jun', 'g_p25_mar',
       'g_p25_may', 'g_p25_nov', 'g_p25_oct', 'g_p25_sep', 'ndvi_p25_apr',
       'ndvi_p25_aug', 'ndvi_p25_dec', 'ndvi_p25_feb', 'ndvi_p25_jan',
       'ndvi_p25_jul', 'ndvi_p25_jun', 'ndvi_p25_mar', 'ndvi_p25_may',
       'ndvi_p25_nov', 'ndvi_p25_oct', 'ndvi_p25_sep', 'nir_p25_apr',
       'nir_p25_aug', 'nir_p25_dec', 'nir_p25_feb', 'nir_p25_jan',
       'nir_p25_jul', 'nir_p25_jun', 'nir_p25_mar', 'nir_p25_may',
       'nir_p25_nov', 'nir_p25_oct', 'nir_p25_sep', 'r_p25_apr',
       'r_p25_aug', 'r_p25_dec', 'r_p25_feb', 'r_p25_jan', 'r_p25_jul',
       'r_p25_jun', 'r_p25_mar', 'r_p25_may', 'r_p25_nov', 'r_p25_oct',
       'r_p25_sep', 'swir1_1_p25_apr', 'swir1_1_p25_aug',
       'swir1_1_p25_dec', 'swir1_1_p25_feb', 'swir1_1_p25_jan',
       'swir1_1_p25_jul', 'swir1_1_p25_jun', 'swir1_1_p25_mar',
       'swir1_1_p25_may', 'swir1_1_p25_nov', 'swir1_1_p25_oct',
       'swir1_1_p25_sep', 'swir1_p25_apr', 'swir1_p25_aug',
       'swir1_p25_dec', 'swir1_p25_feb', 'swir1_p25_jan', 'swir1_p25_jul',
       'swir1_p25_jun', 'swir1_p25_mar', 'swir1_p25_may', 'swir1_p25_nov',
       'swir1_p25_oct', 'swir1_p25_sep', 'swir2_p25_apr', 'swir2_p25_aug',
       'swir2_p25_dec', 'swir2_p25_feb', 'swir2_p25_jan', 'swir2_p25_jul',
       'swir2_p25_jun', 'swir2_p25_mar', 'swir2_p25_may', 'swir2_p25_nov',
       'swir2_p25_oct', 'swir2_p25_sep', 'msavi_p25_apr', 'msavi_p25_aug',
       'msavi_p25_dec', 'msavi_p25_feb', 'msavi_p25_jan', 'msavi_p25_jul',
       'msavi_p25_jun', 'msavi_p25_mar', 'msavi_p25_may', 'msavi_p25_nov',
       'msavi_p25_oct', 'msavi_p25_sep', 'nbr_p25_apr', 'nbr_p25_aug',
       'nbr_p25_dec', 'nbr_p25_feb', 'nbr_p25_jan', 'nbr_p25_jul',
       'nbr_p25_jun', 'nbr_p25_mar', 'nbr_p25_may', 'nbr_p25_nov',
       'nbr_p25_oct', 'nbr_p25_sep', 'ndmi_p25_apr', 'ndmi_p25_aug',
       'ndmi_p25_dec', 'ndmi_p25_feb', 'ndmi_p25_jan', 'ndmi_p25_jul',
       'ndmi_p25_jun', 'ndmi_p25_mar', 'ndmi_p25_may', 'ndmi_p25_nov',
       'ndmi_p25_oct', 'ndmi_p25_sep', 'ndwi_p25_apr', 'ndwi_p25_aug',
       'ndwi_p25_dec', 'ndwi_p25_feb', 'ndwi_p25_jan', 'ndwi_p25_jul',
       'ndwi_p25_jun', 'ndwi_p25_mar', 'ndwi_p25_may', 'ndwi_p25_nov',
       'ndwi_p25_oct', 'ndwi_p25_sep', 'savi_p25_apr', 'savi_p25_aug',
       'savi_p25_dec', 'savi_p25_feb', 'savi_p25_jan', 'savi_p25_jul',
       'savi_p25_jun', 'savi_p25_mar', 'savi_p25_may', 'savi_p25_nov',
       'savi_p25_oct', 'savi_p25_sep', 'merit_hand', 'merit_upstream',
       'monthly_precipitation_apr', 'monthly_precipitation_aug',
       'monthly_precipitation_dec', 'monthly_precipitation_feb',
       'monthly_precipitation_jan', 'monthly_precipitation_jul',
       'monthly_precipitation_jun', 'monthly_precipitation_mar',
       'monthly_precipitation_may', 'monthly_precipitation_nov',
       'monthly_precipitation_oct', 'monthly_precipitation_sep',
       'modis_evi_apr', 'modis_evi_aug', 'modis_evi_dec', 'modis_evi_feb',
       'modis_evi_jan', 'modis_evi_jul', 'modis_evi_jun', 'modis_evi_mar',
       'modis_evi_may', 'modis_evi_nov', 'modis_evi_oct', 'modis_evi_sep',
       'lst_day_1km_p5_apr', 'lst_day_1km_p5_aug', 'lst_day_1km_p5_dec',
       'lst_day_1km_p5_feb', 'lst_day_1km_p5_jan', 'lst_day_1km_p5_jul',
       'lst_day_1km_p5_jun', 'lst_day_1km_p5_mar', 'lst_day_1km_p5_may',
       'lst_day_1km_p5_nov', 'lst_day_1km_p5_oct', 'lst_day_1km_p5_sep',
       'lst_day_1km_p50_apr', 'lst_day_1km_p50_aug',
       'lst_day_1km_p50_dec', 'lst_day_1km_p50_feb',
       'lst_day_1km_p50_jan', 'lst_day_1km_p50_jul',
       'lst_day_1km_p50_jun', 'lst_day_1km_p50_mar',
       'lst_day_1km_p50_may', 'lst_day_1km_p50_nov',
       'lst_day_1km_p50_oct', 'lst_day_1km_p50_sep',
       'lst_day_1km_p95_apr', 'lst_day_1km_p95_aug',
       'lst_day_1km_p95_dec', 'lst_day_1km_p95_feb',
       'lst_day_1km_p95_jan', 'lst_day_1km_p95_jul',
       'lst_day_1km_p95_jun', 'lst_day_1km_p95_mar',
       'lst_day_1km_p95_may', 'lst_day_1km_p95_nov',
       'lst_day_1km_p95_oct', 'lst_day_1km_p95_sep',
       'lst_night_1km_p5_apr', 'lst_night_1km_p5_aug',
       'lst_night_1km_p5_dec', 'lst_night_1km_p5_feb',
       'lst_night_1km_p5_jan', 'lst_night_1km_p5_jul',
       'lst_night_1km_p5_jun', 'lst_night_1km_p5_mar',
       'lst_night_1km_p5_may', 'lst_night_1km_p5_nov',
       'lst_night_1km_p5_oct', 'lst_night_1km_p5_sep',
       'lst_night_1km_p50_apr', 'lst_night_1km_p50_aug',
       'lst_night_1km_p50_dec', 'lst_night_1km_p50_feb',
       'lst_night_1km_p50_jan', 'lst_night_1km_p50_jul',
       'lst_night_1km_p50_jun', 'lst_night_1km_p50_mar',
       'lst_night_1km_p50_may', 'lst_night_1km_p50_nov',
       'lst_night_1km_p50_oct', 'lst_night_1km_p50_sep',
       'lst_night_1km_p95_apr', 'lst_night_1km_p95_aug',
       'lst_night_1km_p95_dec', 'lst_night_1km_p95_feb',
       'lst_night_1km_p95_jan', 'lst_night_1km_p95_jul',
       'lst_night_1km_p95_jun', 'lst_night_1km_p95_mar',
       'lst_night_1km_p95_may', 'lst_night_1km_p95_nov',
       'lst_night_1km_p95_oct', 'lst_night_1km_p95_sep', 'soc_b0',
       'soc_b10', 'soc_b30', 'soc_b60', 'srtm', 'grid_id', 'OC', 'log_OC']

In [ ]:
list2 = ['B_landsat2000', 'G_landsat2000', 'Id_landsat2000',
       'NIR_landsat2000', 'R_landsat2000', 'SWIR1_landsat2000',
       'SWIR2_landsat2000', 'B_landsat2014', 'G_landsat2014',
       'Id_landsat2014', 'NIR_landsat2014', 'R_landsat2014',
       'SWIR1_landsat2014', 'SWIR2_landsat2014', 'pixel_qa_landsat2014',
       'B_landsat2024', 'G_landsat2024', 'NIR_landsat2024',
       'R_landsat2024', 'SWIR1_landsat2024', 'SWIR2_landsat2024',
       'pixel_qa_landsat2024', 'b_p25_apr', 'b_p25_aug', 'b_p25_dec',
       'b_p25_feb', 'b_p25_jan', 'b_p25_jul', 'b_p25_jun', 'b_p25_mar',
       'b_p25_may', 'b_p25_nov', 'b_p25_oct', 'b_p25_sep', 'evi_p25_apr',
       'evi_p25_aug', 'evi_p25_dec', 'evi_p25_feb', 'evi_p25_jan',
       'evi_p25_jul', 'evi_p25_jun', 'evi_p25_mar', 'evi_p25_may',
       'evi_p25_nov', 'evi_p25_oct', 'evi_p25_sep', 'g_p25_apr',
       'g_p25_aug', 'g_p25_dec', 'g_p25_feb', 'g_p25_jan', 'g_p25_jul',
       'g_p25_jun', 'g_p25_mar', 'g_p25_may', 'g_p25_nov', 'g_p25_oct',
       'g_p25_sep', 'ndvi_p25_apr', 'ndvi_p25_aug', 'ndvi_p25_dec',
       'ndvi_p25_feb', 'ndvi_p25_jan', 'ndvi_p25_jul', 'ndvi_p25_jun',
       'ndvi_p25_mar', 'ndvi_p25_may', 'ndvi_p25_nov', 'ndvi_p25_oct',
       'ndvi_p25_sep', 'nir_p25_apr', 'nir_p25_aug', 'nir_p25_dec',
       'nir_p25_feb', 'nir_p25_jan', 'nir_p25_jul', 'nir_p25_jun',
       'nir_p25_mar', 'nir_p25_may', 'nir_p25_nov', 'nir_p25_oct',
       'nir_p25_sep', 'r_p25_apr', 'r_p25_aug', 'r_p25_dec', 'r_p25_feb',
       'r_p25_jan', 'r_p25_jul', 'r_p25_jun', 'r_p25_mar', 'r_p25_may',
       'r_p25_nov', 'r_p25_oct', 'r_p25_sep', 'swir1_1_p25_apr',
       'swir1_1_p25_aug', 'swir1_1_p25_dec', 'swir1_1_p25_feb',
       'swir1_1_p25_jan', 'swir1_1_p25_jul', 'swir1_1_p25_jun',
       'swir1_1_p25_mar', 'swir1_1_p25_may', 'swir1_1_p25_nov',
       'swir1_1_p25_oct', 'swir1_1_p25_sep', 'swir1_p25_apr',
       'swir1_p25_aug', 'swir1_p25_dec', 'swir1_p25_feb', 'swir1_p25_jan',
       'swir1_p25_jul', 'swir1_p25_jun', 'swir1_p25_mar', 'swir1_p25_may',
       'swir1_p25_nov', 'swir1_p25_oct', 'swir1_p25_sep', 'swir2_p25_apr',
       'swir2_p25_aug', 'swir2_p25_dec', 'swir2_p25_feb', 'swir2_p25_jan',
       'swir2_p25_jul', 'swir2_p25_jun', 'swir2_p25_mar', 'swir2_p25_may',
       'swir2_p25_nov', 'swir2_p25_oct', 'swir2_p25_sep', 'msavi_p25_apr',
       'msavi_p25_aug', 'msavi_p25_dec', 'msavi_p25_feb', 'msavi_p25_jan',
       'msavi_p25_jul', 'msavi_p25_jun', 'msavi_p25_mar', 'msavi_p25_may',
       'msavi_p25_nov', 'msavi_p25_oct', 'msavi_p25_sep', 'nbr_p25_apr',
       'nbr_p25_aug', 'nbr_p25_dec', 'nbr_p25_feb', 'nbr_p25_jan',
       'nbr_p25_jul', 'nbr_p25_jun', 'nbr_p25_mar', 'nbr_p25_may',
       'nbr_p25_nov', 'nbr_p25_oct', 'nbr_p25_sep', 'ndmi_p25_apr',
       'ndmi_p25_aug', 'ndmi_p25_dec', 'ndmi_p25_feb', 'ndmi_p25_jan',
       'ndmi_p25_jul', 'ndmi_p25_jun', 'ndmi_p25_mar', 'ndmi_p25_may',
       'ndmi_p25_nov', 'ndmi_p25_oct', 'ndmi_p25_sep', 'ndwi_p25_apr',
       'ndwi_p25_aug', 'ndwi_p25_dec', 'ndwi_p25_feb', 'ndwi_p25_jan',
       'ndwi_p25_jul', 'ndwi_p25_jun', 'ndwi_p25_mar', 'ndwi_p25_may',
       'ndwi_p25_nov', 'ndwi_p25_oct', 'ndwi_p25_sep', 'savi_p25_apr',
       'savi_p25_aug', 'savi_p25_dec', 'savi_p25_feb', 'savi_p25_jan',
       'savi_p25_jul', 'savi_p25_jun', 'savi_p25_mar', 'savi_p25_may',
       'savi_p25_nov', 'savi_p25_oct', 'savi_p25_sep', 'B_p25_aug',
       'G_p25_aug', 'NIR_p25_aug', 'R_p25_aug', 'SWIR1_p25_aug',
       'SWIR2_p25_aug', 'B_p25_nov', 'G_p25_nov', 'NIR_p25_nov',
       'R_p25_nov', 'SWIR1_p25_nov', 'SWIR2_p25_nov', 'B_p25_jun',
       'G_p25_jun', 'NIR_p25_jun', 'R_p25_jun', 'SWIR1_p25_jun',
       'SWIR2_p25_jun', 'B_p25_mar', 'G_p25_mar', 'NIR_p25_mar',
       'R_p25_mar', 'SWIR1_p25_mar', 'SWIR2_p25_mar', 'MSAVI_p25_jun',
       'NBR_p25_jun', 'NDMI_p25_jun', 'NDVI_p25_jun', 'NDWI_p25_jun',
       'SAVI_p25_jun', 'MSAVI_p25_mar', 'NBR_p25_mar', 'NDMI_p25_mar',
       'NDVI_p25_mar', 'NDWI_p25_mar', 'SAVI_p25_mar', 'modis_evi_apr',
       'modis_evi_aug', 'modis_evi_dec', 'modis_evi_feb', 'modis_evi_jan',
       'modis_evi_jul', 'modis_evi_jun', 'modis_evi_mar', 'modis_evi_may',
       'modis_evi_nov', 'modis_evi_oct', 'modis_evi_sep',
       'lst_day_1km_p5_apr', 'lst_day_1km_p5_aug', 'lst_day_1km_p5_dec',
       'lst_day_1km_p5_feb', 'lst_day_1km_p5_jan', 'lst_day_1km_p5_jul',
       'lst_day_1km_p5_jun', 'lst_day_1km_p5_mar', 'lst_day_1km_p5_may',
       'lst_day_1km_p5_nov', 'lst_day_1km_p5_oct', 'lst_day_1km_p5_sep',
       'lst_day_1km_p50_apr', 'lst_day_1km_p50_aug',
       'lst_day_1km_p50_dec', 'lst_day_1km_p50_feb',
       'lst_day_1km_p50_jan', 'lst_day_1km_p50_jul',
       'lst_day_1km_p50_jun', 'lst_day_1km_p50_mar',
       'lst_day_1km_p50_may', 'lst_day_1km_p50_nov',
       'lst_day_1km_p50_oct', 'lst_day_1km_p50_sep',
       'lst_day_1km_p95_apr', 'lst_day_1km_p95_aug',
       'lst_day_1km_p95_dec', 'lst_day_1km_p95_feb',
       'lst_day_1km_p95_jan', 'lst_day_1km_p95_jul',
       'lst_day_1km_p95_jun', 'lst_day_1km_p95_mar',
       'lst_day_1km_p95_may', 'lst_day_1km_p95_nov',
       'lst_day_1km_p95_oct', 'lst_day_1km_p95_sep',
       'lst_night_1km_p5_apr', 'lst_night_1km_p5_aug',
       'lst_night_1km_p5_dec', 'lst_night_1km_p5_feb',
       'lst_night_1km_p5_jan', 'lst_night_1km_p5_jul',
       'lst_night_1km_p5_jun', 'lst_night_1km_p5_mar',
       'lst_night_1km_p5_may', 'lst_night_1km_p5_nov',
       'lst_night_1km_p5_oct', 'lst_night_1km_p5_sep',
       'lst_night_1km_p50_apr', 'lst_night_1km_p50_aug',
       'lst_night_1km_p50_dec', 'lst_night_1km_p50_feb',
       'lst_night_1km_p50_jan', 'lst_night_1km_p50_jul',
       'lst_night_1km_p50_jun', 'lst_night_1km_p50_mar',
       'lst_night_1km_p50_may', 'lst_night_1km_p50_nov',
       'lst_night_1km_p50_oct', 'lst_night_1km_p50_sep',
       'lst_night_1km_p95_apr', 'lst_night_1km_p95_aug',
       'lst_night_1km_p95_dec', 'lst_night_1km_p95_feb',
       'lst_night_1km_p95_jan', 'lst_night_1km_p95_jul',
       'lst_night_1km_p95_jun', 'lst_night_1km_p95_mar',
       'lst_night_1km_p95_may', 'lst_night_1km_p95_nov',
       'lst_night_1km_p95_oct', 'lst_night_1km_p95_sep', 'merit_upstream',
       'merit_hand', 'monthly_precipitation_apr',
       'monthly_precipitation_aug', 'monthly_precipitation_dec',
       'monthly_precipitation_feb', 'monthly_precipitation_jan',
       'monthly_precipitation_jul', 'monthly_precipitation_jun',
       'monthly_precipitation_mar', 'monthly_precipitation_may',
       'monthly_precipitation_nov', 'monthly_precipitation_oct',
       'monthly_precipitation_sep', 'srtm', 'soc_b0', 'soc_b10',
       'soc_b30', 'soc_b60']

In [ ]:
df2.shape

(16231, 317)

In [ ]:
len(list1)

317

In [ ]:
len(list2)

329

In [ ]:
#compare list1 and list2, find what's missing
list3 = list(set(list2) - set(list1))

In [ ]:
df.c

In [ ]:
list3

['evi_p25_sep',
 'evi_p25_mar',
 'evi_p25_dec',
 'evi_p25_jun',
 'evi_p25_jan',
 'evi_p25_feb',
 'Id_landsat2000',
 'pixel_qa_landsat2014',
 'evi_p25_may',
 'pixel_qa_landsat2024',
 'evi_p25_oct',
 'evi_p25_jul',
 'evi_p25_aug',
 'evi_p25_apr',
 'Id_landsat2014',
 'evi_p25_nov']

In [ ]:
df.drop(list3, axis = 1, inplace = True)

In [ ]:
df.shape

(27889, 313)

In [ ]:
df_new = pd.read_csv('train_corrected.csv')

In [ ]:
df_new.columns.values

array(['Unnamed: 0', 'B_landsat2000', 'G_landsat2000', 'Id_landsat2000',
       'NIR_landsat2000', 'R_landsat2000', 'SWIR1_landsat2000',
       'SWIR2_landsat2000', 'B_landsat2014', 'G_landsat2014',
       'Id_landsat2014', 'NIR_landsat2014', 'R_landsat2014',
       'SWIR1_landsat2014', 'SWIR2_landsat2014', 'pixel_qa_landsat2014',
       'B_landsat2024', 'G_landsat2024', 'NIR_landsat2024',
       'R_landsat2024', 'SWIR1_landsat2024', 'SWIR2_landsat2024',
       'pixel_qa_landsat2024', 'b_p25_apr', 'b_p25_aug', 'b_p25_dec',
       'b_p25_feb', 'b_p25_jan', 'b_p25_jul', 'b_p25_jun', 'b_p25_mar',
       'b_p25_may', 'b_p25_nov', 'b_p25_oct', 'b_p25_sep', 'g_p25_apr',
       'g_p25_aug', 'g_p25_dec', 'g_p25_feb', 'g_p25_jan', 'g_p25_jul',
       'g_p25_jun', 'g_p25_mar', 'g_p25_may', 'g_p25_nov', 'g_p25_oct',
       'g_p25_sep', 'ndvi_p25_apr', 'ndvi_p25_aug', 'ndvi_p25_dec',
       'ndvi_p25_feb', 'ndvi_p25_jan', 'ndvi_p25_jul', 'ndvi_p25_jun',
       'ndvi_p25_mar', 'ndvi_p25_may', 'ndvi

In [ ]:
df.to_csv('corrected_train')

In [ ]:


# Modified pred_mlr function
pred_mlr <- function(tvar, id, year, t.m, eml.cf, in_dir=in_dir, out_dir=out_dir) {
  # Set fixed parameters for simplified version
  multiplier = ifelse(tvar=="clay_tot_psa" | tvar=="sand_tot_psa", 1,
                     ifelse(tvar=="db_od", 100, 10))
  min.md = 1

  out.tif = paste0(out_dir, id, "/", tvar, "_M_", year, "_1km_T", id, ".tif")
  if(!file.exists(out.tif)){

    g30m = readRDS("/content/my_spatial_omo.rds")

# Extract only the data part for processing
    g30m_data = g30m@data  # Extract attribute table

    # Clean the data: Replace infinite or NaN values with NA
    g30m_data[, t.m$features] = do.call(data.frame, lapply(g30m_data[, t.m$features], function(x)
      replace(x, is.infinite(x) | is.nan(x), NA)))

    # Keep complete cases
    cc = complete.cases(g30m_data)

    if(sum(cc) > 0) {
      # Retain spatial structure for complete cases
      g1kx = g30m[cc, ]

      # Make predictions
      pred = predict(t.m, newdata=g30m_data)
      out.p <- as.matrix(as.data.frame(mlr::getStackedBaseLearnerPredictions(t.m, newdata=g1kx)))

      # Calculate model error
      g1kx$model.error <- sqrt(matrixStats::rowSds(out.p, na.rm=TRUE)^2 * eml.cf) * multiplier
      g1kx$model.error <- ifelse(g1kx$model.error < min.md, min.md, g1kx$model.error)

      # Set predictions
      g1kx$pred = ifelse(pred$data$response < 0, 0, pred$data$response) * multiplier

      # Convert to raster and write output
      pred_rast <- terra::rast(g1kx["pred"])
      terra::writeRaster(pred_rast, out.tif, datatype="INT1U", NAflag=255,
                        overwrite=TRUE, gdal=c("COMPRESS=DEFLATE"))

      error_rast <- terra::rast(g1kx["model.error"])
      terra::writeRaster(error_rast, gsub("_M_", "_md_", out.tif), datatype="INT1U",
                        NAflag=255, overwrite=TRUE, gdal=c("COMPRESS=DEFLATE"))
    }
  }
}

# Simplified pred_lst function
pred_lst <- function(tvar, id, year, t.m, eml.cf, in_dir="./input/"){
  if(missing(t.m)){
    t.m <- readRDS.gz(paste0("./modelsT/eml.m_", tvar, ".rds"))
    m.train = t.m$learner.model$super.model$learner.model$model
    m.terms = all.vars(t.m$learner.model$super.model$learner.model$terms)
    eml.MSE0 = matrixStats::rowSds(as.matrix(m.train[,m.terms[-1]]), na.rm=TRUE)^2
    eml.MSE = deviance(t.m$learner.model$super.model$learner.model)/df.residual(t.m$learner.model$super.model$learner.model)
    ## correction factor:
    eml.cf = eml.MSE/mean(eml.MSE0, na.rm = TRUE)
  }

  # Call pred_mlr for a single year
  pred_mlr(tvar=tvar, id=id, year=year, t.m=t.m, eml.cf=eml.cf, in_dir=in_dir)
}





In [ ]:
import pandas as pd

df = pd.read_csv('LUCAS-SOIL-2018.csv')
df2 = pd.read_csv('train_new_data_lucas.csv')

In [ ]:
df.shape

(18984, 27)

In [ ]:
df2.shape

(16231, 317)

In [ ]:
df3 = pd.read_csv('/content/training_data_with_oc.csv')
df4 = pd.read_csv('/content/train_new_data.csv')

In [ ]:
df3.shape

(18950, 270)

In [ ]:
df3.head()

,POINTID,b_p25_apr,b_p25_aug,b_p25_dec,b_p25_feb,b_p25_jan,b_p25_jul,b_p25_jun,b_p25_mar,b_p25_may,...,lst_night_1km_p95_jul,lst_night_1km_p95_jun,lst_night_1km_p95_mar,lst_night_1km_p95_may,lst_night_1km_p95_nov,lst_night_1km_p95_oct,lst_night_1km_p95_sep,grid_id,OC,log_oc
0,26521970,73.0,94.000000,54.0,62.0,52.0,100.000000,94.000000,79.0,90.000000,...,14698.31630,14625.56965,14266.94750,14561.37446,14356.46909,14535.52667,14639.65587,17,19.0,2.944439
1,26581768,81.0,93.512821,57.0,63.0,56.0,96.081081,98.473684,82.9,94.000000,...,14643.26902,14595.24241,14226.50218,14474.49292,14343.06827,14490.77015,14566.39479,10,28.2,3.339322
2,26581792,71.0,78.900000,50.0,56.0,50.0,83.400000,76.000000,62.0,79.851852,...,14646.83274,14598.28401,14226.01187,14477.58198,14340.91547,14490.62623,14567.43445,11,33.5,3.511545
3,26581954,84.0,100.000000,55.0,65.0,54.0,100.000000,101.000000,78.0,94.000000,...,14731.48541,14652.37410,14268.59340,14583.85403,14351.66889,14541.37144,14654.02184,17,13.7,2.617396
4,26601784,67.5,72.000000,50.0,55.0,47.0,77.633333,74.500000,59.0,72.000000,...,14646.71518,14598.08634,14226.51355,14477.34631,14341.86492,14491.24258,14567.94414,11,52.1,3.953165


In [ ]:
df.columns

Index(['Depth', 'POINTID', 'pH_CaCl2', 'pH_H2O', 'EC', 'OC', 'CaCO3', 'P', 'N',
       'K', 'OC (20-30 cm)', 'CaCO3 (20-30 cm)', 'Ox_Al', 'Ox_Fe', 'NUTS_0',
       'NUTS_1', 'NUTS_2', 'NUTS_3', 'TH_LAT', 'TH_LONG', 'SURVEY_DATE',
       'Elev', 'LC', 'LU', 'LC0_Desc', 'LC1_Desc', 'LU1_Desc'],
      dtype='object')

In [ ]:
df = df[['Depth', 'POINTID', 'pH_CaCl2', 'pH_H2O', 'EC', 'OC', 'CaCO3', 'P', 'N',
       'K', 'OC (20-30 cm)', 'CaCO3 (20-30 cm)', 'Ox_Al', 'Ox_Fe', 'NUTS_0']]

In [ ]:
merged_df = pd.merge(df, df2, on='POINTID', how='inner')

In [ ]:
merged_df.shape

(16231, 331)

In [ ]:
merged_df.columns.values

array(['Depth', 'POINTID', 'pH_CaCl2', 'pH_H2O', 'EC', 'OC_x', 'CaCO3',
       'P', 'N', 'K', 'OC (20-30 cm)', 'CaCO3 (20-30 cm)', 'Ox_Al',
       'Ox_Fe', 'NUTS_0', 'B_p25_aug', 'G_p25_aug', 'NIR_p25_aug',
       'R_p25_aug', 'SWIR1_p25_aug', 'SWIR2_p25_aug', 'B_p25_jun',
       'G_p25_jun', 'NIR_p25_jun', 'R_p25_jun', 'SWIR1_p25_jun',
       'SWIR2_p25_jun', 'B_p25_mar', 'G_p25_mar', 'NIR_p25_mar',
       'R_p25_mar', 'SWIR1_p25_mar', 'SWIR2_p25_mar', 'B_p25_nov',
       'G_p25_nov', 'NIR_p25_nov', 'R_p25_nov', 'SWIR1_p25_nov',
       'SWIR2_p25_nov', 'MSAVI_p25_jun', 'NBR_p25_jun', 'NDMI_p25_jun',
       'NDVI_p25_jun', 'NDWI_p25_jun', 'SAVI_p25_jun', 'MSAVI_p25_mar',
       'NBR_p25_mar', 'NDMI_p25_mar', 'NDVI_p25_mar', 'NDWI_p25_mar',
       'SAVI_p25_mar', 'B_landsat2000', 'G_landsat2000',
       'NIR_landsat2000', 'R_landsat2000', 'SWIR1_landsat2000',
       'SWIR2_landsat2000', 'B_landsat2014', 'G_landsat2014',
       'NIR_landsat2014', 'R_landsat2014', 'SWIR1_landsat2014',
   

In [ ]:
merged_df.drop(['OC_y',  'OC_x',], axis = 1, inplace = True)


In [ ]:
merged_df.to_csv('merged_df.csv', index= False)

In [ ]:
merged_df['CaCO3 (20-30 cm)']

In [ ]:
#replace all rows with '<load with NA
import numpy as np
merged_df.replace('< LOD', np.nan, inplace=True)

In [ ]:
merged_df.replace('NA', np.nan, inplace = True)

In [ ]:
merged_df['CaCO3'].head(10)

,CaCO3
0,3
1,1
2,1
3,3
4,2
5,NaN
6,NaN
7,96
8,86
9,3


In [ ]:
merged_df.to_csv('merged_df_corrected_na.csv', index = False)

In [ ]:
import pandas as pd
import numpy as np


df = pd.read_csv('merged_df_corrected_na.csv')
df.shape

<ipython-input-1-6cee87e03f4e>:5: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_df_corrected_na.csv')


(16231, 329)

In [ ]:
df.head()

,Depth,POINTID,pH_CaCl2,pH_H2O,EC,CaCO3,P,N,K,OC (20-30 cm),...,lst_night_1km_p95_nov,lst_night_1km_p95_oct,lst_night_1km_p95_sep,soc_b0,soc_b10,soc_b30,soc_b60,srtm,grid_id,log_OC
0,0-20 cm,47862690,4.1,4.81,8.73,3,NaN,1.1,101.9,NaN,...,14059.23944,14309.44722,14502.43316,5,5,2,1,303,9200,2.517696
1,0-20 cm,47882704,4.1,4.93,5.06,1,NaN,1.3,51.2,NaN,...,14047.62693,14301.61708,14503.55864,8,6,2,0,375,9201,2.815409
2,0-20 cm,47982688,4.1,4.85,12.53,1,12.3,3.1,114.8,NaN,...,14064.18535,14316.16361,14510.87544,4,4,2,1,266,9200,3.860730
3,0-20 cm,48022702,5.5,5.80,21.10,3,NaN,2.0,165.8,NaN,...,14053.75032,14310.76105,14515.42913,5,5,2,1,303,9201,3.335770
4,0-20 cm,48062708,6.1,6.48,10.89,2,NaN,2.2,42.1,NaN,...,14050.60983,14309.78697,14518.89076,7,7,2,1,327,9201,2.965273


In [ ]:
df.replace('<  LOD', np.nan, inplace=True)

In [ ]:
columns_to_convert = ['pH_CaCl2', 'pH_H2O', 'EC', 'CaCO3', 'P', 'N', 'K']

for column in columns_to_convert:
    df[column] = df[column].astype(float)


In [ ]:

#create log of these columns and create new columns log_
for column in columns_to_convert:
    df[f'log_{column}'] = np.log(df[column])

In [ ]:
df.head(3)

,Depth,POINTID,pH_CaCl2,pH_H2O,EC,CaCO3,P,N,K,OC (20-30 cm),...,srtm,grid_id,log_OC,log_pH_CaCl2,log_pH_H2O,log_EC,log_CaCO3,log_P,log_N,log_K
0,0-20 cm,47862690,4.1,4.81,8.73,3.0,NaN,1.1,101.9,NaN,...,303,9200,2.517696,1.410987,1.570697,2.166765,1.098612,NaN,0.095310,4.623992
1,0-20 cm,47882704,4.1,4.93,5.06,1.0,NaN,1.3,51.2,NaN,...,375,9201,2.815409,1.410987,1.595339,1.621366,0.000000,NaN,0.262364,3.935740
2,0-20 cm,47982688,4.1,4.85,12.53,1.0,12.3,3.1,114.8,NaN,...,266,9200,3.860730,1.410987,1.578979,2.528126,0.000000,2.509599,1.131402,4.743191


In [ ]:
df.to_csv('merged_df_corrected_na_log.csv', index = False)